# Silane-Methane Competition in Sub-Neptunes (Hakim et al. 2025)

## Import libraries

In [ ]:
import logging

import numpy as np
import optimistix as optx

from atmodeller import (
    InteriorAtmosphere,
    Planet,
    SolverParameters,
    Species,
    SpeciesCollection,
    debug_logger,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models

from atmodeller.thermodata import IronWustiteBuffer

logger = debug_logger()
logger.setLevel(logging.INFO)
# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm

from scipy.interpolate import interp1d
import os

from matplotlib.ticker import ScalarFormatter
formatter = ScalarFormatter(useMathText=False)
formatter.set_scientific(False)
formatter.set_useOffset(False)
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

# Uncomment below to install labellines if not already installed
#!pip install matplotlib-label-lines

from labellines import labelLine, labelLines

## Elemental Abundances

In [ ]:
# Palme and O'Neill (2014) Treatise on Geochemistry - Table 3

SiO2_mantlemasspercent_palme14: float = 45.4
total_mantlemasspercent_palme14: float = 98.41
core_mass_fraction: float = 0.327  # typical values used are between 0.325 - 0.33 
Si_massfraction_palme14: float = round(SiO2_mantlemasspercent_palme14 / 
                                            total_mantlemasspercent_palme14 * 
                                            (1 - core_mass_fraction) / (28.0855+2*15.999) * 28.0855, 3)

O_massfraction_palme14: float = Si_massfraction_palme14 / 28.0855 * 2 * 15.999


# Lodders et al. (2009) Springer Book Chapter - Table 8 (in wt%)

H_masspercent_lodders09: float = 73.9
He_masspercent_lodders09: float = 24.69
C_masspercent_lodders09: float = 0.22
N_masspercent_lodders09: float = 0.07
O_masspercent_lodders09: float = 0.63
Si_masspercent_lodders09: float = 0.07


# Lodders et al. (2009) Springer Book Chapter - Table 6 (log-normalalized abundances relative to H)

H_logN: float = 12
He_logN: float = 10.93
C_logN: float = 8.39
N_logN: float = 7.86
O_logN: float = 8.73
Si_logN: float = 7.53

Primitive composition of the Earth's mantle \
Table 3 - Palme and O'Neill (2014) Treatise on Geochemistry

| Component  | Mass % |
|------------|--------|
| MgO        | 36.77  |
| Al2O3      | 4.49   |
| SiO2       | 45.4   |
| CaO        | 3.65   |
| FeO(t)     | 8.1    |
| Total      | 98.41  |
| Mg#        | 0.890  |


Present-day solar composition \
Table 8 - Lodders et al. (2009) Springer book chapter 

| Element    | Mass % |
|------------|--------|
| H (=X)     | 73.9   |
| He (=Y)    | 24.69  |
| O          | 0.63   |
| C          | 0.22   |
| Ne         | 0.17   |
| Fe         | 0.12   |
| N          | 0.07   |
| Si         | 0.07   |
| Mg         | 0.06   |
| S          | 0.03   |
| others     | 0.04   |
| total (=Z) | 1.41   |

Table 6 - Lodders et al. (2009) Springer book chapter 

| Element | A (log N(H) = 12) |
|---------|-------------------|
| H       | 12                |
| He      | 10.93             |
| C       | 8.39              |
| N       | 7.86              |
| O       | 8.73              |
| Ne      | 8.05              |
| Na      | 6.29              |
| Mg      | 7.54              |
| Al      | 6.46              |
| Si      | 7.53              |
| P       | 5.45              |
| S       | 7.16              |
| Cl      | 5.25              |
| Ar      | 6.5               |
| K       | 5.11              |
| Ca      | 6.31              |
| Ti      | 4.93              |
| V       | 3.99              |
| Fe      | 7.46              |


## Example A. Compare fugacity coefficients

### H2 fugacity coefficients

In [ ]:
eos_models = get_eos_models()

pressures = np.logspace(0, 6, num=200)
temperature = 3000

fc_holland = []
fc_beattie = []
fc_shi = []
fc_chabrier = []
fc_zhang = []

for pressure in pressures:
    fc_holland.append(eos_models['H2_cork_cs_holland91'].fugacity_coefficient(temperature=temperature, pressure=pressure))
    fc_beattie.append(eos_models['H2_beattie_holley58'].fugacity_coefficient(temperature=temperature, pressure=pressure))
    fc_shi.append(eos_models['H2_shi92'].fugacity_coefficient(temperature=temperature, pressure=pressure))
    fc_chabrier.append(eos_models['H2_chabrier21'].fugacity_coefficient(temperature=temperature, pressure=pressure))
    fc_zhang.append(eos_models['H2_zhang09'].fugacity_coefficient(temperature=temperature, pressure=pressure))

fig, ax1 = plt.subplots(1, figsize=(4,4), tight_layout='True')

ax1.plot(pressures/1e3, fc_holland, color='orange', lw=2, label=r'Holland')
ax1.plot(pressures/1e3, fc_beattie, color='blue', lw=2, label=r'Holley')
ax1.plot(pressures/1e3, fc_shi, color='pink', lw=2, label=r'Shi')
ax1.plot(pressures/1e3, fc_chabrier, color='green', lw=2, label=r'Chabrier')
ax1.plot(pressures/1e3, fc_zhang, color='cyan', lw=2, label=r'Zhang')

ax1.set_title('H2 Fugacity Coefficients at %d K'%temperature)

ax1.set_ylim([1e-2,1e8])

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Total Pressure [kbar]', fontsize=14)
ax1.set_ylabel(r'Fugacity Coefficient', fontsize=14)

xvals = [0.8, 0.55, 0.22, 0.104, 0.045]
#labelLines(plt.gca().get_lines())

ax1.legend(ncol=1, loc='upper left')
plt.savefig('exampleA_H2fugcoeff_%dK.pdf'%temperature, bbox_inches='tight')
plt.savefig('exampleA_H2fugcoeff_%dK.png'%temperature, bbox_inches='tight')
plt.show()

### SiO and SiH4 fugacity coefficients

In [ ]:
eos_models = get_eos_models()

pressures = np.logspace(0, 5, num=200)
temperature = 3000

fc_wang_H4Si = []
fc_connolly_RK_OSi = []
fc_reid_RK_H4Si = []

for pressure in pressures:
    fc_wang_H4Si.append(eos_models['H4Si_wang18'].fugacity_coefficient(temperature=temperature, pressure=pressure))
    fc_connolly_RK_OSi.append(eos_models['OSi_rk49_connolly16'].fugacity_coefficient(temperature=temperature, pressure=pressure))
    fc_reid_RK_H4Si.append(eos_models['H4Si_rk49_reid87'].fugacity_coefficient(temperature=temperature, pressure=pressure))

fig, ax1 = plt.subplots(1, figsize=(4,4), tight_layout='True')


ax1.plot(pressures/1e3, fc_wang_H4Si, color='red', lw=2, label=r'SiH$_4$ (Virial, Wang+2018/This Work)')
ax1.plot(pressures/1e3, fc_reid_RK_H4Si, color='red', lw=2, ls=':', label=r'SiH$_4$ (RK49, Reid 1987)')
ax1.plot(pressures/1e3, fc_connolly_RK_OSi, color='black', ls='-', lw=2, label=r'SiO (RK49, Connolly 2016)')

ax1.set_title('SiH4 and SiO Fugacity Coefficients at %d K'%temperature)

ax1.set_ylim([1e-2,1e10])

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Total Pressure [kbar]', fontsize=14)
ax1.set_ylabel(r'Fugacity Coefficient', fontsize=14)

ax1.legend(ncol=1, loc='upper left')
plt.savefig('exampleA_SiOSiH4fugcoeff_%dK.pdf'%temperature, bbox_inches='tight')
plt.savefig('exampleA_SiOSiH4fugcoeff_%dK.png'%temperature, bbox_inches='tight')
plt.show()

## Example B. Minimum Working Example with Ideal Gas and No Solubility

In [ ]:
number_of_realisations = 50
surface_temperature = 3000
# Mass and radius
planet_mass = 6.7*5.972e24 # kg
surface_radius = 1.65*6371000 # metre
# 1.65 Earth radii in metre for 6.7 Earth masses
# M-R relation from Hakim et al. (2018) Icarus
# Update mantle melt fraction below
mantle_melt_fraction = 1.0

hmps = np.logspace(-1, 0, num=number_of_realisations) # wt%
h_kgs = hmps / 100 * planet_mass # kg
si_kgs_solar: float = h_kgs * 0.07 / 73.9 # Lodders et al. (2009) 
si_kg_magma: float = 0.145 * planet_mass # Si = 14.5 wt% Palme & O'Neill (2014)
o_kg_magma: float = si_kg_magma * 2 * 15.999 / 28.0855  # O = 2 * SiO2 / Si

planet = Planet(
    surface_temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

H2O_g = Species.create_gas("H2O")
H2_g = Species.create_gas("H2")
O2_g = Species.create_gas("O2")
OSi_g = Species.create_gas("OSi")
H4Si_g = Species.create_gas("H4Si")
O2Si_l = Species.create_condensed("O2Si", state="l")

species = SpeciesCollection((H2O_g, H2_g, O2_g, OSi_g, H4Si_g, O2Si_l))

mass_constraints = {
    "Si": si_kg_magma,
    "O": o_kg_magma,
    "H": h_kgs,
}

model_ideal_nosol = InteriorAtmosphere(species)

model_ideal_nosol.solve(
    planet=planet,
    mass_constraints=mass_constraints,
)
output_ideal_nosol = model_ideal_nosol.output

output_ideal_nosol.quick_look()

output_ideal_nosol.to_excel("exampleB_ideal_nosol")

## Example C. Minimum Working Example with Real Gas and Solubility

In [ ]:
number_of_realisations = 50
surface_temperature = 3000
# Mass and radius
planet_mass = 6.7*5.972e24 # kg
surface_radius = 1.65*6371000 # metre
# 1.65 Earth radii in metre for 6.7 Earth masses
# M-R relation from Hakim et al. (2018) Icarus
# Update mantle melt fraction below
mantle_melt_fraction = 1.0

hmps = np.logspace(-1, 0, num=number_of_realisations) # wt%
h_kgs = hmps / 100 * planet_mass # kg
si_kgs_solar: float = h_kgs * 0.07 / 73.9
si_kg_magma: float = 0.145 * planet_mass # Si = 14.5 wt% Palme & O'Neill (2014)
o_kg_magma: float = si_kg_magma * 2 * 15.999 / 28.0855  # O = 2 * SiO2 / Si

planet = Planet(
    surface_temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

eos_models = get_eos_models()
sol_models = get_solubility_models()

H2O_g = Species.create_gas("H2O", activity=eos_models["H2O_cork_holland98"], 
                           solubility=sol_models["H2O_peridotite_sossi23"])
H2_g = Species.create_gas("H2", activity=eos_models["H2_chabrier21"], 
                          solubility=sol_models["H2_basalt_hirschmann12"])
O2_g = Species.create_gas("O2", activity=eos_models["O2_cs_shi92"])
OSi_g = Species.create_gas("OSi", activity=eos_models['OSi_rk49_connolly16'])
H4Si_g = Species.create_gas("H4Si", activity=eos_models['H4Si_wang18'])
O2Si_l = Species.create_condensed("O2Si", state="l")

species = SpeciesCollection((H2O_g, H2_g, O2_g, OSi_g, H4Si_g, O2Si_l))

mass_constraints = {
    "Si": si_kg_magma,
    "O": o_kg_magma,
    "H": h_kgs,
}

model_real_sol = InteriorAtmosphere(species)

solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

initial_log_number_density = output_ideal_nosol.log_number_density 

model_real_sol.solve(
    planet=planet,
    mass_constraints=mass_constraints,
    initial_log_number_density=initial_log_number_density,
    solver_parameters=solver_parameters,
)
output_real_sol = model_real_sol.output

output_real_sol.quick_look()

output_real_sol.to_excel("exampleC_real_sol")

# Model Setup

## Planet Parameters

In [ ]:
# Mass and radius of TOI-421b
planet_mass = 6.7 * 5.972e24 # kg
MEB_radius = 1.65 * 6371000 # metre
atm_radius = 2.64 * 6371000 # metre
# MEB radius = 1.65 Earth radii in metre for 6.7 Earth masses (for atm_radius = 2.64 Earth radii)
# M-R relation from Hakim et al. (2018) Icarus

# Temperature of TOI-421b
MEB_temperature = 3000 # K
atm_temperature = 920 # K

## Elemental Budgets

In [ ]:
number_of_realisations = 50

# For envelope composition setup
hmps = np.logspace(-1, 0, num=number_of_realisations) # wt% H
h_kgs = hmps / 100 * planet_mass # kg

si_kg_magma: float = Si_massfraction_palme14 * planet_mass 
o_kg_magma: float = O_massfraction_palme14 * planet_mass 

# Lodders et al. (2009) Springer book chapter Table 8
si_kgs_solar = h_kgs * Si_masspercent_lodders09 / H_masspercent_lodders09
o_kgs_solar = h_kgs * O_masspercent_lodders09 / H_masspercent_lodders09
c_kgs_solar = h_kgs * C_masspercent_lodders09 / H_masspercent_lodders09
n_kgs_solar = h_kgs * N_masspercent_lodders09 / H_masspercent_lodders09
he_kgs_solar = h_kgs * He_masspercent_lodders09 / H_masspercent_lodders09

# For atmosphere composition setup
atm_scalings = np.logspace(-10, 0, num=number_of_realisations) 

h_kg = 1 / 100 * planet_mass # kg for 1 wt% H

# Lodders et al. (2009) Springer book chapter Table 8
si_kg_solar = h_kg * Si_masspercent_lodders09 / H_masspercent_lodders09
o_kg_solar = h_kg * O_masspercent_lodders09 / H_masspercent_lodders09
c_kg_solar = h_kg * C_masspercent_lodders09 / H_masspercent_lodders09
n_kg_solar = h_kg * N_masspercent_lodders09 / H_masspercent_lodders09
he_kg_solar = h_kg * He_masspercent_lodders09 / H_masspercent_lodders09

## Chemical Species Setup

In [ ]:
eos_models = get_eos_models()
sol_models = get_solubility_models()

H2O_g = Species.create_gas("H2O")
H2O_gs = Species.create_gas("H2O", solubility=sol_models["H2O_peridotite_sossi23"])
H2O_rgs = Species.create_gas("H2O", activity=eos_models["H2O_cork_holland98"],
                             solubility=sol_models["H2O_peridotite_sossi23"])

H2_g = Species.create_gas("H2")
H2_gs = Species.create_gas("H2", solubility=sol_models["H2_basalt_hirschmann12"])
H2_rgs = Species.create_gas("H2", activity=eos_models["H2_chabrier21"],
                            solubility=sol_models["H2_basalt_hirschmann12"])

O2_g = Species.create_gas("O2")
O2_rg = Species.create_gas("O2", activity=eos_models["O2_cs_shi92"])

OSi_g = Species.create_gas("OSi")
OSi_rg = Species.create_gas("OSi", activity=eos_models['OSi_rk49_connolly16'])

H4Si_g = Species.create_gas("H4Si")
H4Si_rg = Species.create_gas("H4Si", activity=eos_models['H4Si_wang18'])

O2Si_l = Species.create_condensed("O2Si", state="l")
O2Si_bqz = Species.create_condensed("O2Si", state="bqz")

CO2_g = Species.create_gas("CO2")
CO2_gs = Species.create_gas("CO2", solubility=sol_models["CO2_basalt_dixon95"])
CO2_rgs = Species.create_gas("CO2", solubility=sol_models["CO2_basalt_dixon95"],
                             activity=eos_models["CO2_cs_shi92"])

CO_g = Species.create_gas("CO")
CO_gs = Species.create_gas("CO", solubility=sol_models["CO_basalt_yoshioka19"])
CO_rgs = Species.create_gas("CO", solubility=sol_models["CO_basalt_yoshioka19"],
                            activity=eos_models["CO_cs_shi92"])

CH4_g = Species.create_gas("CH4")
CH4_gs = Species.create_gas("CH4", solubility=sol_models["CH4_basalt_ardia13"])
CH4_rgs = Species.create_gas("CH4", solubility=sol_models["CH4_basalt_ardia13"],
                             activity=eos_models["CH4_cs_shi92"])

N2_g = Species.create_gas("N2")
N2_gs = Species.create_gas("N2", solubility=sol_models["N2_basalt_libourel03"])
N2_rgs = Species.create_gas("N2", solubility=sol_models["N2_basalt_libourel03"],
                            activity=eos_models["N2_cs_saxena87"])

NH3_g = Species.create_gas("NH3")
NH3_rg = Species.create_gas("H3N", activity=eos_models["H3N_rk49_reid87"])

He_g = Species.create_gas("He")
He_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
He_rgs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"],
                            activity=eos_models["He_chabrier21"])


species_HOSi_magma_nosol_ideal = SpeciesCollection((H2O_g, H2_g, O2_g, OSi_g, H4Si_g, O2Si_l))
species_HOSi_magma_sol_ideal = SpeciesCollection((H2O_gs, H2_gs, O2_g, OSi_g, H4Si_g, O2Si_l))
species_HOSi_magma_sol_real = SpeciesCollection((H2O_rgs, H2_rgs, O2_rg, OSi_rg, H4Si_rg, O2Si_l))


species_HHeCNOSi_magma_nosol_ideal = SpeciesCollection((H2O_g, H2_g, O2_g, OSi_g, H4Si_g, O2Si_l, 
                                                        CO2_g, CO_g, CH4_g, N2_g, NH3_g, He_g))
species_HHeCNOSi_magma_sol_ideal = SpeciesCollection((H2O_gs, H2_gs, O2_g, OSi_g, H4Si_g, O2Si_l, 
                                                      CO2_gs, CO_gs, CH4_gs, N2_gs, NH3_g, He_gs))
species_HHeCNOSi_magma_sol_real = SpeciesCollection((H2O_rgs, H2_rgs, O2_rg, OSi_rg, H4Si_rg, O2Si_l,
                                                     CO2_rgs, CO_rgs, CH4_rgs, N2_rgs, NH3_rg, He_rgs))

species_HHeCNOSi_atmosphere = SpeciesCollection((H2O_g, H2_g, O2_g, OSi_g, H4Si_g, O2Si_bqz, 
                                                 CO2_g, CO_g, CH4_g, N2_g, NH3_g, He_g))


# H-O-Si

## Envelope Composition 

In [ ]:
# Update accreted gas metallicity below
init_metallicity = 1 # 1x solar

# Update mantle melt fraction below
mantle_melt_fraction = 1

planet = Planet(surface_temperature = MEB_temperature, planet_mass = planet_mass, 
                surface_radius = MEB_radius, mantle_melt_fraction=mantle_melt_fraction)

mass_constraints = {
    "Si": si_kg_magma * mantle_melt_fraction + si_kgs_solar * init_metallicity,
    "O": o_kg_magma * mantle_melt_fraction + o_kgs_solar * init_metallicity,
    "H": h_kgs,
}

# Magma - No Solubility - Ideal Gas
model_magma_nosol_ideal = InteriorAtmosphere(species_HOSi_magma_nosol_ideal)
model_magma_nosol_ideal.solve(
    planet=planet,
    mass_constraints=mass_constraints,
)
output_magma_nosol_ideal = model_magma_nosol_ideal.output
output_magma_nosol_ideal.quick_look()
output_magma_nosol_ideal.to_excel(f"HOSi_magma_nosol_ideal_{init_metallicity}xsolar")

# Magma - Solubility - Ideal Gas
model_magma_sol_ideal = InteriorAtmosphere(species_HOSi_magma_sol_ideal)
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)
initial_log_number_density = output_magma_nosol_ideal.log_number_density 
model_magma_sol_ideal.solve(
    planet=planet,
    mass_constraints=mass_constraints,
    initial_log_number_density=initial_log_number_density,
    solver_parameters=solver_parameters,
)
output_magma_sol_ideal = model_magma_sol_ideal.output
output_magma_sol_ideal.quick_look()
output_magma_sol_ideal.to_excel(f"HOSi_magma_sol_ideal_{init_metallicity}xsolar")

# Magma - Solubility - Real Gas
model_magma_sol_real = InteriorAtmosphere(species_HOSi_magma_sol_real)
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver, multistart=0)
initial_log_number_density = output_magma_sol_ideal.log_number_density 
model_magma_sol_real.solve(
    planet=planet,
    mass_constraints=mass_constraints,
    initial_log_number_density=initial_log_number_density,
    solver_parameters=solver_parameters,
)
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel(f"HOSi_magma_sol_real_{init_metallicity}xsolar")

# H-He-C-N-O-Si

## Envelope Composition 

In [ ]:
mantle_melt_fractions = [1, 0.3, 0.1, 0.03, 0.01] # mantle melt fraction
init_metallicitys = [1, 3, 10, 30, 100] # metallicity in x solar units

for mantle_melt_fraction in mantle_melt_fractions:

    planet = Planet(surface_temperature=MEB_temperature, planet_mass=planet_mass, 
                    mantle_melt_fraction=mantle_melt_fraction, surface_radius=MEB_radius)

    for init_metallicity in init_metallicitys:

        mass_constraints = {
            "H": h_kgs,
            "He": he_kgs_solar,
            "C": init_metallicity * c_kgs_solar,
            "N": init_metallicity * n_kgs_solar,
            "Si": init_metallicity * si_kgs_solar + mantle_melt_fraction * si_kg_magma,
            "O": init_metallicity * o_kgs_solar +  mantle_melt_fraction * o_kg_magma, 
            }
        
        # Magma - No solubility - Ideal Gas
        model_magma_nosol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_nosol_ideal)
        model_magma_nosol_ideal.solve(
            planet=planet,
            mass_constraints=mass_constraints,
        )
        output_magma_nosol_ideal = model_magma_nosol_ideal.output
        output_magma_nosol_ideal.quick_look()
        output_magma_nosol_ideal.to_excel(f"HHeCNOSi_magma_nosol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}")

        # Magma - Solubility - Ideal Gas
        model_magma_sol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_sol_ideal)
        solver = optx.LevenbergMarquardt
        solver_parameters = SolverParameters(solver=solver)
        initial_log_number_density = output_magma_nosol_ideal.log_number_density 
        model_magma_sol_ideal.solve(
            planet=planet,
            mass_constraints=mass_constraints,
            initial_log_number_density=initial_log_number_density,
            solver_parameters=solver_parameters,
        )
        output_magma_sol_ideal = model_magma_sol_ideal.output
        output_magma_sol_ideal.quick_look()
        output_magma_sol_ideal.to_excel(f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}")

        # Magma - Solubility - Real Gas
        model_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_magma_sol_real)
        solver = optx.LevenbergMarquardt
        solver_parameters = SolverParameters(solver=solver, multistart=0)
        initial_log_number_density = output_magma_sol_ideal.log_number_density
        model_magma_sol_real.solve(
            planet=planet,
            mass_constraints=mass_constraints,
            initial_log_number_density=initial_log_number_density,
            solver_parameters=solver_parameters,
        )
        output_magma_sol_real = model_magma_sol_real.output
        output_magma_sol_real.quick_look()
        output_magma_sol_real.to_excel(f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}")


## Atmosphere Composition for 1 H wt% with SiO2 cond.

In [ ]:
# setup atmodeller planet object as atmosphere with no magma (melt fraction = 0)
planet = Planet(surface_temperature=atm_temperature, planet_mass=planet_mass, 
                mantle_melt_fraction=0, surface_radius=atm_radius)

mantle_melt_fractions = [1, 0.3, 0.1, 0.03, 0.01] # mantle melt fraction
init_metallicitys = [1, 3, 10, 30, 100] # metallicity in x solar units

for mantle_melt_fraction in mantle_melt_fractions:
    for init_metallicity in init_metallicitys:
        
        # Magma - No solubility - Ideal Gas
        filename = f"HHeCNOSi_magma_nosol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

        H_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"][49]
        He_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"][49]
        C_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"][49]
        N_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"][49]
        Si_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"][49]
        O_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"][49]
        tot_pressure_magma_nosol_ideal = pd.read_excel(filename, sheet_name="atmosphere")["pressure"][49]

        mass_constraints = {
            "H": H_mass_atm_magma_nosol_ideal * atm_scalings,
            "He": He_mass_atm_magma_nosol_ideal * atm_scalings,
            "C": C_mass_atm_magma_nosol_ideal * atm_scalings,
            "N": N_mass_atm_magma_nosol_ideal * atm_scalings,
            "O": O_mass_atm_magma_nosol_ideal * atm_scalings,
            "Si": Si_mass_atm_magma_nosol_ideal * atm_scalings,
        }

        model_vertical_magma_nosol_ideal = InteriorAtmosphere(species_HHeCNOSi_atmosphere)
        model_vertical_magma_nosol_ideal.solve(
            planet=planet,
            mass_constraints=mass_constraints,
        )
        output_vertical_magma_nosol_ideal = model_vertical_magma_nosol_ideal.output
        output_vertical_magma_nosol_ideal.quick_look()
        output_vertical_magma_nosol_ideal.to_excel(f"HHeCNOSi_vertical_magma_nosol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz")

        # Magma - Solubility - Ideal Gas
        filename = f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

        H_mass_atm_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"][49]
        He_mass_atm_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"][49]
        C_mass_atm_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"][49]
        N_mass_atm_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"][49]
        Si_mass_atm_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"][49]
        O_mass_atm_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"][49]
        tot_pressure_magma_sol_ideal = pd.read_excel(filename, sheet_name="atmosphere")["pressure"][49]

        mass_constraints = {
            "H": H_mass_atm_magma_sol_ideal * atm_scalings,
            "He": He_mass_atm_magma_sol_ideal * atm_scalings,
            "C": C_mass_atm_magma_sol_ideal * atm_scalings,
            "N": N_mass_atm_magma_sol_ideal * atm_scalings,
            "O": O_mass_atm_magma_sol_ideal * atm_scalings,
            "Si": Si_mass_atm_magma_sol_ideal * atm_scalings,
        } 

        model_vertical_magma_sol_ideal = InteriorAtmosphere(species_HHeCNOSi_atmosphere)
        model_vertical_magma_sol_ideal.solve(
            planet=planet,
            mass_constraints=mass_constraints,
        )
        output_vertical_magma_sol_ideal = model_vertical_magma_sol_ideal.output
        output_vertical_magma_sol_ideal.quick_look()
        output_vertical_magma_sol_ideal.to_excel(f"HHeCNOSi_vertical_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz")

        # Magma - Solubility - Real Gas
        filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

        H_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"][49]
        He_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"][49]
        C_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"][49]
        N_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"][49]
        Si_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"][49]
        O_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"][49]
        tot_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="atmosphere")["pressure"][49]

        mass_constraints = {
            "H": H_mass_atm_magma_sol_real * atm_scalings,
            "He": He_mass_atm_magma_sol_real * atm_scalings,
            "C": C_mass_atm_magma_sol_real * atm_scalings,
            "N": N_mass_atm_magma_sol_real * atm_scalings,
            "O": O_mass_atm_magma_sol_real * atm_scalings,
            "Si": Si_mass_atm_magma_sol_real * atm_scalings,
        }

        model_vertical_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_atmosphere)

        model_vertical_magma_sol_real.solve(
            planet=planet,
            mass_constraints=mass_constraints,
        )
        output_vertical_magma_sol_real = model_vertical_magma_sol_real.output

        output_vertical_magma_sol_real.quick_look()

        output_vertical_magma_sol_real.to_excel(f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz")

for init_metallicity in init_metallicitys:
    # No Magma (Accreted Gas Metallicity)
    mass_constraints = {
        "H": h_kg * atm_scalings,
        "He": he_kg_solar * atm_scalings,
        "C": init_metallicity * c_kg_solar * atm_scalings,
        "N": init_metallicity * n_kg_solar * atm_scalings,
        "O": init_metallicity * o_kg_solar * atm_scalings,
        "Si": init_metallicity * si_kg_solar * atm_scalings,
    }

    model_vertical_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_atmosphere)
    model_vertical_magma_sol_real.solve(
        planet=planet,
        mass_constraints=mass_constraints,
    )
    output_vertical_magma_sol_real = model_vertical_magma_sol_real.output
    output_vertical_magma_sol_real.quick_look()
    output_vertical_magma_sol_real.to_excel(f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt0_1wtH_bqz")

# Plots published with Hakim et al. (2025)

## Setup plot colors

In [ ]:
color_tot = 'black'
color_H2 = 'orange'
color_H2O = 'blue'
color_SiH4 = 'red'
color_SiO = 'brown'
color_O2 = 'limegreen'
color_CO2 = 'cyan'
color_CO = 'magenta'
color_CH4 = 'purple'
color_N2 = 'green'
color_NH3 = 'pink'
color_He = 'gray'
color_H = 'orange'
color_C = 'cyan'
color_N = 'green'
color_O = 'blue'
color_Si = 'red'

## A. H-O-Si Envelope Composition

### Retrieve Data

In [ ]:
init_metallicity = 1 # 1x solar

### ideal + no solubility

filename = f"HOSi_magma_nosol_ideal_{init_metallicity}xsolar.xlsx"

H2_pressure_magma_nosol_ideal = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_magma_nosol_ideal = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_magma_nosol_ideal = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_magma_nosol_ideal = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_magma_nosol_ideal = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
tot_pressure_magma_nosol_ideal = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

H_moles_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
O_moles_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
Si_moles_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_moles_melt_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_H")["dissolved_moles"]
O_moles_melt_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_O")["dissolved_moles"]
Si_moles_melt_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_Si")["dissolved_moles"]

H_moles_tot_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_H")["total_moles"]
O_moles_tot_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_O")["total_moles"]
Si_moles_tot_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_Si")["total_moles"]

H_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
O_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
Si_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_melt_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_H")["dissolved_mass"]
O_mass_melt_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_O")["dissolved_mass"]
Si_mass_melt_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_Si")["dissolved_mass"]

H_mass_tot_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
O_mass_tot_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
Si_mass_tot_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_magma_nosol_ideal = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_magma_nosol_ideal = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_magma_nosol_ideal = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_magma_nosol_ideal = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_magma_nosol_ideal = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]

### ideal + solubility

filename = f"HOSi_magma_sol_ideal_{init_metallicity}xsolar.xlsx"

H2_pressure_magma_sol_ideal = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_magma_sol_ideal = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_magma_sol_ideal = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_magma_sol_ideal = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_magma_sol_ideal = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
tot_pressure_magma_sol_ideal = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

H_moles_atm_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
O_moles_atm_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
Si_moles_atm_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_moles_melt_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_H")["dissolved_moles"]
O_moles_melt_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_O")["dissolved_moles"]
Si_moles_melt_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_Si")["dissolved_moles"]

H_moles_tot_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_H")["total_moles"]
O_moles_tot_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_O")["total_moles"]
Si_moles_tot_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_Si")["total_moles"]

H_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
O_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
Si_mass_atm_magma_nosol_ideal = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_melt_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_H")["dissolved_mass"]
O_mass_melt_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_O")["dissolved_mass"]
Si_mass_melt_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_Si")["dissolved_mass"]

H_mass_tot_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
O_mass_tot_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
Si_mass_tot_magma_sol_ideal = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_magma_sol_ideal = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_magma_sol_ideal = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_magma_sol_ideal = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_magma_sol_ideal = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_magma_sol_ideal = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]

### real + solubility

filename = f"HOSi_magma_sol_real_{init_metallicity}xsolar.xlsx"

H2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
tot_pressure_magma_sol_real = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

H_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
O_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
Si_moles_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["dissolved_moles"]
O_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["dissolved_moles"]
Si_moles_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["dissolved_moles"]

H_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["total_moles"]
O_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["total_moles"]
Si_moles_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["total_moles"]

H_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
O_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
Si_mass_atm_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["dissolved_mass"]
O_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["dissolved_mass"]
Si_mass_melt_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["dissolved_mass"]

H_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
O_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
Si_mass_tot_magma_sol_real = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_magma_sol_real = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_magma_sol_real = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_magma_sol_real = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_magma_sol_real = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_magma_sol_real = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]

### Fig. 3 - Plot MEB Partial Pressures

In [ ]:
fig, (ax0, ax1) = plt.subplots(2, figsize=(6,6), tight_layout='True')


ax0.plot(hmps, tot_pressure_magma_sol_real, color=color_tot, lw=4, ls= '-', label='Total')
ax0.plot(hmps, H2_pressure_magma_sol_real, color=color_H2, lw=3, ls= '-', label='H$_2$')
ax0.plot(hmps, H2O_pressure_magma_sol_real, color=color_H2O, lw=3, ls= '-', label='H$_2$O')
ax0.plot(hmps, SiH4_pressure_magma_sol_real, color=color_SiH4, lw=3, ls= '-', label='SiH$_4$')
ax0.plot(hmps, SiO_pressure_magma_sol_real, color=color_SiO, lw=3, ls= '-', label='SiO')
#ax0.plot(hmps, O2_pressure_magma_sol_real, color=color_O2, lw=3, ls= '-', label='O$_2$')

ax0.set_title(r'(a) Real gases') 
ax0.set_ylim([1e0, 2e5])
ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax0.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax0.set_xticks([0.1, 0.3, 1])
ax0.get_xaxis().set_major_formatter(ScalarFormatter())

ax00 = ax0.twinx()
ax00.set_yscale('log')
ax00.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax00.set_ylim([1e-4, 2e1])

ax1.plot(hmps, tot_pressure_magma_sol_ideal, color=color_tot, lw=4, ls= '-.', label='Total')
ax1.plot(hmps, H2_pressure_magma_sol_ideal, color=color_H2, lw=3, ls= '-.', label='H$_2$')
ax1.plot(hmps, H2O_pressure_magma_sol_ideal, color=color_H2O, lw=3, ls= '-.', label='H$_2$O')
ax1.plot(hmps, SiH4_pressure_magma_sol_ideal, color=color_SiH4, lw=3, ls= '-.', label='SiH$_4$')
ax1.plot(hmps, SiO_pressure_magma_sol_ideal, color=color_SiO, lw=3, ls= '-.', label='SiO')
#ax1.plot(hmps, O2_pressure_magma_sol_ideal, color=color_O2, lw=3, ls= '-.', label='O$_2$')

ax1.set_title(r'(b) Ideal gases') 
ax1.set_ylim([1e0, 2e5])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax1.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax1.set_xticks([0.1, 0.3, 1])
ax1.get_xaxis().set_major_formatter(ScalarFormatter())

ax11 = ax1.twinx()
ax11.set_yscale('log')
ax11.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax11.set_ylim([1e-4, 2e1])

# ax2.plot(hmps, tot_pressure_magma_nosol_ideal, color=color_tot, lw=4, ls= '--')
# ax2.plot(hmps, H2_pressure_magma_nosol_ideal, color=color_H2, lw=3, ls= '--')
# ax2.plot(hmps, H2O_pressure_magma_nosol_ideal, color=color_H2O, lw=3, ls= '--')
# ax2.plot(hmps, SiH4_pressure_magma_nosol_ideal, color=color_SiH4, lw=3, ls= '--')
# ax2.plot(hmps, SiO_pressure_magma_nosol_ideal, color=color_SiO, lw=3, ls= '--')
# #ax2.plot(hmps, O2_pressure_magma_nosol_ideal, color=color_O2, lw=3, ls= '--')

# ax2.set_title(r'(c) Ideal gas + No solubility')
# ax2.set_ylim([1e0, 1e6])
# ax2.set_xscale('log')
# ax2.set_yscale('log')
# ax2.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
# ax2.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)

# ax22 = ax2.twinx()
# ax22.set_yscale('log')
# ax22.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
# ax22.set_ylim([1e-4, 1e2])

# ax0.legend(ncol=5,  bbox_to_anchor=(0.01, 0.81),
#           fancybox=True, loc='lower left', fontsize=9)

labelLines(ax0.get_lines())
labelLines(ax1.get_lines())

fig.suptitle(f'Envelope Composition at MEB (H$-$O$-$Si system)', fontsize=16)

# plt.savefig(f"HOSi_envelope_{init_metallicity}xsolar.pdf", bbox_inches='tight')
# plt.savefig(f"HOSi_envelope_{init_metallicity}xsolar.png", bbox_inches='tight')
plt.savefig(f"HOSi_envelope.pdf", bbox_inches='tight')
plt.savefig(f"HOSi_envelope.png", bbox_inches='tight')
plt.show()

### Fig. 4 - Plot Fugacity Coefficients

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(6,3), tight_layout='True')

l1, = ax1.plot(hmps, H2_fc_magma_sol_real, color=color_H2, lw=3, ls= '-', label='H$_2$')
l2, = ax1.plot(hmps, H2O_fc_magma_sol_real, color=color_H2O, lw=3, ls= '-', label='H$_2$O')
l3, = ax1.plot(hmps, SiH4_fc_magma_sol_real, color=color_SiH4, lw=3, ls= '-', label='SiH$_4$')
l4, = ax1.plot(hmps, SiO_fc_magma_sol_real, color=color_SiO, lw=3, ls= '-', label='SiO')
l5, = ax1.plot(hmps, O2_fc_magma_sol_real, color=color_O2, lw=3, ls='-', label='O$_2$')

ax1.set_title(f'Real Gas Fugacity Coefficients (H$-$O$-$Si system)')

ax1.set_ylim([8e-2, 2e2])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax1.set_ylabel(r'Fugacity Coefficient', fontsize=12)
ax1.set_xticks([0.1, 0.3, 1])
ax1.get_xaxis().set_major_formatter(ScalarFormatter())

labelLines(plt.gca().get_lines())

#ax1.legend(ncol=2)

# plt.savefig(f"HOSi_fugacitycoeff_{init_metallicity}xsolar.pdf", bbox_inches='tight')
# plt.savefig(f"HOSi_fugacitycoeff_{init_metallicity}xsolar.png", bbox_inches='tight')
plt.savefig(f"HOSi_fugacitycoeff.pdf", bbox_inches='tight')
plt.savefig(f"HOSi_fugacitycoeff.png", bbox_inches='tight')
plt.show()

### Fig. 5 - Plot H Partitioning in Magma

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(6,3), tight_layout='True')

ax1.plot(hmps, 100 * H_mass_melt_magma_sol_real/H_mass_tot_magma_sol_real, color=color_tot, lw=3, ls= '-', label='Real')
ax1.plot(hmps, 100 * H_mass_melt_magma_sol_ideal/H_mass_tot_magma_sol_ideal, color=color_tot, lw=3, ls= '-.', label='Ideal')

ax1.set_title(f'Hydrogen Solubility in Magma (H$-$O$-$Si system)')

ax1.set_ylim([-5, 105])
ax1.set_xscale('log')
ax1.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax1.set_ylabel(r'Partitioning in Magma [%]', fontsize=12)
ax1.set_xticks([0.1, 0.3, 1])
ax1.get_xaxis().set_major_formatter(ScalarFormatter())

#ax1.legend(ncol=1, fontsize=14)

labelLines(ax1.get_lines())

# plt.savefig(f"HOSi_Hmagma_{init_metallicity}xsolar.pdf", bbox_inches='tight')
# plt.savefig(f"HOSi_Hmagma_{init_metallicity}xsolar.png", bbox_inches='tight')
plt.savefig(f"HOSi_Hmagma.pdf", bbox_inches='tight')
plt.savefig(f"HOSi_Hmagma.png", bbox_inches='tight')
plt.show()

## B. H-He-C-N-O-Si Envelope composition

### Retrieve Data

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 1

# 1x solar
init_metallicity = 1 # 1x solar

filename = f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]

He_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_ideal_1xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_ideal_1xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]


### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_1xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 10x solar 
init_metallicity = 10 # 10x solar

filename = f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_ideal_10xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_ideal_10xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]


### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_10xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 100x solar
init_metallicity = 100 # 100x solar

filename = f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_ideal_100xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_ideal_100xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]


### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_100xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 0.01

# 1x solar
init_metallicity = 1 # 1x solar

filename = f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_iseal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_ideal_1xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_ideal_1xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]


### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_1xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 10x solar
init_metallicity = 10 # 10x solar

filename = f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_ideal_10xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_ideal_10xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]


### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_10xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 100x solar
init_metallicity = 100 # 100x solar

filename = f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_ideal_100xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_ideal_100xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]


### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_100xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_mass_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
He_mass_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
O_mass_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
C_mass_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
N_mass_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
Si_mass_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]

H_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

### Fig. 6 -  Ideal vs Real - Envelope Composition, Fugacity Coefficients, Volatile Partitioning in Magma

In [ ]:
# 1x solar 
init_metallicity = 1 # 1x solar

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2,2, figsize=(10,6), tight_layout='True')

ax0.plot(hmps, CO_pressure_ideal_1xsolar_melt100, color=color_CO, lw=2, ls= '-.', label='CO')
ax0.plot(hmps, tot_pressure_ideal_1xsolar_melt100, color=color_tot, lw=3, ls= '-.', label='Total')
ax0.plot(hmps, H2_pressure_ideal_1xsolar_melt100, color=color_H2, lw=2, ls= '-.', label='H$_2$')
ax0.plot(hmps, H2O_pressure_ideal_1xsolar_melt100, color=color_H2O, lw=2, ls= '-.', label='H$_2$O')
ax0.plot(hmps, O2_pressure_ideal_1xsolar_melt100, color=color_O2, lw=2, ls= '-.', label='O$_2$')
ax0.plot(hmps, SiH4_pressure_ideal_1xsolar_melt100, color=color_SiH4, lw=2, ls= '-.', label='SiH$_4$')
ax0.plot(hmps, SiO_pressure_ideal_1xsolar_melt100, color=color_SiO, lw=2, ls= '-.', label='SiO')
ax0.plot(hmps, CO2_pressure_ideal_1xsolar_melt100, color=color_CO2, lw=2, ls= '-.', label='CO$_2$')
ax0.plot(hmps, CH4_pressure_ideal_1xsolar_melt100, color=color_CH4, lw=2, ls= '-.', label='CH$_4$')
ax0.plot(hmps, N2_pressure_ideal_1xsolar_melt100, color=color_N2, lw=2, ls= '-.', label='N$_2$')
ax0.plot(hmps, NH3_pressure_ideal_1xsolar_melt100, color=color_NH3, lw=2, ls= '-.', label='NH$_3$')
ax0.plot(hmps, He_pressure_ideal_1xsolar_melt100, color=color_He, lw=2, ls= '-.', label='He')

ax0.set_title(r'(a) Envelope Composition at MEB (Ideal)') 
ax0.set_ylim([1e0, 2e5])
ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax0.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax0.set_xticks([0.1, 0.3, 1])
ax0.get_xaxis().set_major_formatter(ScalarFormatter())

ax1.plot(hmps, CO_pressure_real_1xsolar_melt100, color=color_CO, lw=2, ls= '-', label='CO')
ax1.plot(hmps, tot_pressure_real_1xsolar_melt100, color=color_tot, lw=3, ls= '-', label='Total')
ax1.plot(hmps, H2_pressure_real_1xsolar_melt100, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax1.plot(hmps, H2O_pressure_real_1xsolar_melt100, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax1.plot(hmps, O2_pressure_real_1xsolar_melt100, color=color_O2, lw=2, ls= '-', label='O$_2$')
ax1.plot(hmps, SiH4_pressure_real_1xsolar_melt100, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax1.plot(hmps, SiO_pressure_real_1xsolar_melt100, color=color_SiO, lw=2, ls= '-', label='SiO')
ax1.plot(hmps, He_pressure_real_1xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
ax1.plot(hmps, CO2_pressure_real_1xsolar_melt100, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax1.plot(hmps, CH4_pressure_real_1xsolar_melt100, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax1.plot(hmps, N2_pressure_real_1xsolar_melt100, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax1.plot(hmps, NH3_pressure_real_1xsolar_melt100, color=color_NH3, lw=2, ls= '-', label='NH$_3$')

ax1.set_title(r'(b) Envelope Composition at MEB (Real)') 
ax1.set_ylim([1e0, 2e5])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax1.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax1.set_xticks([0.1, 0.3, 1])
ax1.get_xaxis().set_major_formatter(ScalarFormatter())

ax00 = ax0.twinx()
ax00.set_yscale('log')
ax00.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax00.set_ylim([1e-4, 2e1])

ax11 = ax1.twinx()
ax11.set_yscale('log')
ax11.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax11.set_ylim([1e-4, 2e1])

l1, = ax2.plot(hmps, 100 * (H_mass_tot_real_1xsolar_melt100-H_mass_atm_real_1xsolar_melt100)/H_mass_tot_real_1xsolar_melt100, color=color_H, lw=2, ls= '-', label='H')
l2, = ax2.plot(hmps, 100 * (He_mass_tot_real_1xsolar_melt100-He_mass_atm_real_1xsolar_melt100)/He_mass_tot_real_1xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
l3, = ax2.plot(hmps, 100 * (C_mass_tot_real_1xsolar_melt100-C_mass_atm_real_1xsolar_melt100)/C_mass_tot_real_1xsolar_melt100, color=color_C, lw=2, ls= '-', label='C')
l4, = ax2.plot(hmps, 100 * (N_mass_tot_real_1xsolar_melt100-N_mass_atm_real_1xsolar_melt100)/N_mass_tot_real_1xsolar_melt100, color=color_N, lw=2, ls= '-', label='N')
ax2.plot(hmps, 100 * (H_mass_tot_ideal_1xsolar_melt100-H_mass_atm_ideal_1xsolar_melt100)/H_mass_tot_ideal_1xsolar_melt100, color=color_H, lw=2, ls= '-.')
ax2.plot(hmps, 100 * (He_mass_tot_ideal_1xsolar_melt100-He_mass_atm_ideal_1xsolar_melt100)/He_mass_tot_ideal_1xsolar_melt100, color=color_He, lw=2, ls= '-.')
ax2.plot(hmps, 100 * (C_mass_tot_ideal_1xsolar_melt100-C_mass_atm_ideal_1xsolar_melt100)/C_mass_tot_ideal_1xsolar_melt100, color=color_C, lw=2, ls= '-.')
ax2.plot(hmps, 100 * (N_mass_tot_ideal_1xsolar_melt100-N_mass_atm_ideal_1xsolar_melt100)/N_mass_tot_ideal_1xsolar_melt100, color=color_N, lw=2, ls= '-.')

ax2.set_title(r'(c) Volatile Solubility in Magma (Ideal vs Real)')

ax2.set_ylim([-5, 105])
ax2.set_xscale('log')
ax2.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax2.set_ylabel(r'Partitioning in Magma [%]', fontsize=12)
ax2.set_xticks([0.1, 0.3, 1])
ax2.get_xaxis().set_major_formatter(ScalarFormatter())

ax3.plot(hmps, H2_fc_real_1xsolar_melt100, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax3.plot(hmps, H2O_fc_real_1xsolar_melt100, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax3.plot(hmps, O2_fc_real_1xsolar_melt100, color=color_O2, lw=2, ls= '-', label='O$_2$')
ax3.plot(hmps, SiH4_fc_real_1xsolar_melt100, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax3.plot(hmps, SiO_fc_real_1xsolar_melt100, color=color_SiO, lw=2, ls= '-', label='SiO')
ax3.plot(hmps, He_fc_real_1xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
ax3.plot(hmps, CO2_fc_real_1xsolar_melt100, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax3.plot(hmps, CO_fc_real_1xsolar_melt100, color=color_CO, lw=2, ls= '-', label='CO')
ax3.plot(hmps, CH4_fc_real_1xsolar_melt100, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax3.plot(hmps, N2_fc_real_1xsolar_melt100, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax3.plot(hmps, NH3_fc_real_1xsolar_melt100, color=color_NH3, lw=2, ls= '-', label='NH$_3$')


ax3.set_title(r'(d) Real Gas Fugacity Coefficients')

ax3.set_ylim([8e-2, 1e5])
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax3.set_ylabel(r'Fugacity Coefficient', fontsize=12)
ax3.set_xticks([0.1, 0.3, 1])
ax3.get_xaxis().set_major_formatter(ScalarFormatter())

labelLines(ax0.get_lines())
labelLines(ax1.get_lines())
labelLines(ax2.get_lines())
labelLines(ax3.get_lines())

custom_handles = [Line2D([0], [0], color='black', ls='-.', lw=2, label='Ideal'),
                  Line2D([0], [0], color='black', ls='-', lw=2, label='Real')]

# old_handles, labels = ax2.get_legend_handles_labels()

ax2.legend(handles=custom_handles,  bbox_to_anchor=(0.01, 0.41), loc='upper left',
           ncol=1,  labelspacing=0.1, borderpad=0.2, columnspacing=0.1, framealpha=0.5)

# ax1.legend(ncol=5, labelspacing=0, borderpad=0, columnspacing=0.1, fontsize=9, framealpha=0.2)

# ax3.legend(ncol=5, labelspacing=0, borderpad=0, columnspacing=0.1, fontsize=9, framealpha=0.2)



fig.suptitle(r'Envelope Composition at MEB for Ideal vs Real Gases (H$-$He$-$C$-$N$-$O$-$Si system, 1$\times$ solar)', fontsize=16)

plt.savefig("HHeCNOSi_ideal_real.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_ideal_real.png", bbox_inches='tight')
plt.show()

### Fig. A1 - Full Range - Ideal vs Real - Envelope Composition

In [ ]:
# 1x solar 
init_metallicity = 1 # 1x solar

fig, ((ax0, ax1)) = plt.subplots(1,2, figsize=(10,3), tight_layout='True')

ax0.plot(hmps, tot_pressure_ideal_1xsolar_melt100, color=color_tot, lw=3, ls= '-.', label='Total')
ax0.plot(hmps, H2_pressure_ideal_1xsolar_melt100, color=color_H2, lw=2, ls= '-.', label='H$_2$')
ax0.plot(hmps, H2O_pressure_ideal_1xsolar_melt100, color=color_H2O, lw=2, ls= '-.', label='H$_2$O')
ax0.plot(hmps, O2_pressure_ideal_1xsolar_melt100, color=color_O2, lw=2, ls= '-.', label='O$_2$')
ax0.plot(hmps, SiH4_pressure_ideal_1xsolar_melt100, color=color_SiH4, lw=2, ls= '-.', label='SiH$_4$')
ax0.plot(hmps, SiO_pressure_ideal_1xsolar_melt100, color=color_SiO, lw=2, ls= '-.', label='SiO')
ax0.plot(hmps, CO2_pressure_ideal_1xsolar_melt100, color=color_CO2, lw=2, ls= '-.', label='CO$_2$')
ax0.plot(hmps, CO_pressure_ideal_1xsolar_melt100, color=color_CO, lw=2, ls= '-.', label='CO')
ax0.plot(hmps, CH4_pressure_ideal_1xsolar_melt100, color=color_CH4, lw=2, ls= '-.', label='CH$_4$')
ax0.plot(hmps, N2_pressure_ideal_1xsolar_melt100, color=color_N2, lw=2, ls= '-.', label='N$_2$')
ax0.plot(hmps, NH3_pressure_ideal_1xsolar_melt100, color=color_NH3, lw=2, ls= '-.', label='NH$_3$')
ax0.plot(hmps, He_pressure_ideal_1xsolar_melt100, color=color_He, lw=2, ls= '-.', label='He')

ax0.set_title(r'(a) Envelope Composition at MEB (Ideal)') 
ax0.set_ylim([2e-11, 2e5])
ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax0.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax0.set_xticks([0.1, 0.3, 1])
ax0.get_xaxis().set_major_formatter(ScalarFormatter())

ax1.plot(hmps, tot_pressure_real_1xsolar_melt100, color=color_tot, lw=3, ls= '-', label='Total')
ax1.plot(hmps, H2_pressure_real_1xsolar_melt100, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax1.plot(hmps, H2O_pressure_real_1xsolar_melt100, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax1.plot(hmps, O2_pressure_real_1xsolar_melt100, color=color_O2, lw=2, ls= '-', label='O$_2$')
ax1.plot(hmps, SiH4_pressure_real_1xsolar_melt100, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax1.plot(hmps, SiO_pressure_real_1xsolar_melt100, color=color_SiO, lw=2, ls= '-', label='SiO')
ax1.plot(hmps, He_pressure_real_1xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
ax1.plot(hmps, CO2_pressure_real_1xsolar_melt100, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax1.plot(hmps, CO_pressure_real_1xsolar_melt100, color=color_CO, lw=2, ls= '-', label='CO')
ax1.plot(hmps, CH4_pressure_real_1xsolar_melt100, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax1.plot(hmps, N2_pressure_real_1xsolar_melt100, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax1.plot(hmps, NH3_pressure_real_1xsolar_melt100, color=color_NH3, lw=2, ls= '-', label='NH$_3$')

ax1.set_title(r'(b) Envelope Composition at MEB (Real)') 
ax1.set_ylim([2e-11, 2e5])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax1.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax1.set_xticks([0.1, 0.3, 1])
ax1.get_xaxis().set_major_formatter(ScalarFormatter())

ax00 = ax0.twinx()
ax00.set_yscale('log')
ax00.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax00.set_ylim([2e-15, 2e1])

ax11 = ax1.twinx()
ax11.set_yscale('log')
ax11.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax11.set_ylim([2e-15, 2e1])

labelLines(ax0.get_lines())
labelLines(ax1.get_lines())

fig.suptitle(r'Envelope Composition at MEB for Ideal vs Real Gases (H$-$He$-$C$-$N$-$O$-$Si system, 1$\times$ solar)', fontsize=16)

plt.savefig("HHeCNOSi_ideal_real_fullrange.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_ideal_real_fullrange.png", bbox_inches='tight')
plt.show()

### Fig. 7 - Effects of Mantle Melt Fraction and Metallicity - Envelope Composition

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2,2, figsize=(10,6), tight_layout='True')

ax0.plot(hmps, CO_pressure_real_1xsolar_melt100, color=color_CO, lw=2, ls= '-', label='CO')
ax0.plot(hmps, tot_pressure_real_1xsolar_melt100, color=color_tot, lw=3, ls= '-', label='Total')
ax0.plot(hmps, H2_pressure_real_1xsolar_melt100, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax0.plot(hmps, He_pressure_real_1xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
ax0.plot(hmps, H2O_pressure_real_1xsolar_melt100, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax0.plot(hmps, SiH4_pressure_real_1xsolar_melt100, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax0.plot(hmps, SiO_pressure_real_1xsolar_melt100, color=color_SiO, lw=2, ls= '-', label='SiO')
ax0.plot(hmps, CH4_pressure_real_1xsolar_melt100, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax0.plot(hmps, CO2_pressure_real_1xsolar_melt100, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax0.plot(hmps, N2_pressure_real_1xsolar_melt100, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax0.plot(hmps, NH3_pressure_real_1xsolar_melt100, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax0.plot(hmps, O2_pressure_real_1xsolar_melt100, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax0.set_title(r'(a) 1$\times$ solar, 100% melt')
ax0.set_ylim([1e0, 2e5])
ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax0.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax0.set_xticks([0.1, 0.3, 1])
ax0.get_xaxis().set_major_formatter(ScalarFormatter())

ax1.plot(hmps, tot_pressure_real_100xsolar_melt100, color=color_tot, lw=3, ls= '-', label='Total')
ax1.plot(hmps, H2_pressure_real_100xsolar_melt100, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax1.plot(hmps, He_pressure_real_100xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
ax1.plot(hmps, H2O_pressure_real_100xsolar_melt100, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax1.plot(hmps, SiH4_pressure_real_100xsolar_melt100, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax1.plot(hmps, SiO_pressure_real_100xsolar_melt100, color=color_SiO, lw=2, ls= '-', label='SiO')
ax1.plot(hmps, CH4_pressure_real_100xsolar_melt100, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax1.plot(hmps, CO_pressure_real_100xsolar_melt100, color=color_CO, lw=2, ls= '-', label='CO')
ax1.plot(hmps, CO2_pressure_real_100xsolar_melt100, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax1.plot(hmps, N2_pressure_real_100xsolar_melt100, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax1.plot(hmps, NH3_pressure_real_100xsolar_melt100, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax1.plot(hmps, O2_pressure_real_100xsolar_melt100, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax1.set_title(r'(b) 100$\times$ solar, 100% melt')
ax1.set_ylim([1e0, 2e5])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax1.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax1.set_xticks([0.1, 0.3, 1])
ax1.get_xaxis().set_major_formatter(ScalarFormatter())

ax2.plot(hmps, tot_pressure_real_1xsolar_melt1, color=color_tot, lw=3, ls= '-', label='Total')
ax2.plot(hmps, H2_pressure_real_1xsolar_melt1, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax2.plot(hmps, He_pressure_real_1xsolar_melt1, color=color_He, lw=2, ls= '-', label='He')
ax2.plot(hmps, H2O_pressure_real_1xsolar_melt1, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax2.plot(hmps, SiH4_pressure_real_1xsolar_melt1, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax2.plot(hmps, SiO_pressure_real_1xsolar_melt1, color=color_SiO, lw=2, ls= '-', label='SiO')
ax2.plot(hmps, CH4_pressure_real_1xsolar_melt1, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax2.plot(hmps, CO_pressure_real_1xsolar_melt1, color=color_CO, lw=2, ls= '-', label='CO')
ax2.plot(hmps, CO2_pressure_real_1xsolar_melt1, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax2.plot(hmps, N2_pressure_real_1xsolar_melt1, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax2.plot(hmps, NH3_pressure_real_1xsolar_melt1, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax2.plot(hmps, O2_pressure_real_1xsolar_melt1, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax2.set_title(r'(c) 1$\times$ solar, 1% melt') 
ax2.set_ylim([1e0, 2e5])
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax2.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax2.set_xticks([0.1, 0.3, 1])
ax2.get_xaxis().set_major_formatter(ScalarFormatter())

ax3.plot(hmps, tot_pressure_real_100xsolar_melt1, color=color_tot, lw=3, ls= '-', label='Total')
ax3.plot(hmps, H2_pressure_real_100xsolar_melt1, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax3.plot(hmps, He_pressure_real_100xsolar_melt1, color=color_He, lw=2, ls= '-', label='He')
ax3.plot(hmps, H2O_pressure_real_100xsolar_melt1, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax3.plot(hmps, SiH4_pressure_real_100xsolar_melt1, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax3.plot(hmps, SiO_pressure_real_100xsolar_melt1, color=color_SiO, lw=2, ls= '-', label='SiO')
ax3.plot(hmps, CH4_pressure_real_100xsolar_melt1, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax3.plot(hmps, CO_pressure_real_100xsolar_melt1, color=color_CO, lw=2, ls= '-', label='CO')
ax3.plot(hmps, CO2_pressure_real_100xsolar_melt1, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax3.plot(hmps, N2_pressure_real_100xsolar_melt1, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax3.plot(hmps, NH3_pressure_real_100xsolar_melt1, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax3.plot(hmps, O2_pressure_real_100xsolar_melt1, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax3.set_title(r'(d) 100$\times$ solar, 1% melt') 
ax3.set_ylim([1e0, 2e5])
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax3.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax3.set_xticks([0.1, 0.3, 1])
ax3.get_xaxis().set_major_formatter(ScalarFormatter())

ax00 = ax0.twinx()
ax00.set_yscale('log')
ax00.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax00.set_ylim([1e-4, 2e1])

ax11 = ax1.twinx()
ax11.set_yscale('log')
ax11.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax11.set_ylim([1e-4, 2e1])

ax22 = ax2.twinx()
ax22.set_yscale('log')
ax22.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax22.set_ylim([1e-4, 2e1])

ax33 = ax3.twinx()
ax33.set_yscale('log')
ax33.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax33.set_ylim([1e-4, 2e1])

labelLines(ax0.get_lines())
labelLines(ax1.get_lines())
labelLines(ax2.get_lines())
labelLines(ax3.get_lines())

fig.suptitle(r'Envelope Composition at MEB (H$-$He$-$C$-$N$-$O$-$Si system)', fontsize=16)

plt.savefig("HHeCNOSi_envelope.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_envelope.png", bbox_inches='tight')
plt.show()

### Fig. A2 - Full Range - Effects of Mantle Melt Fraction and Metallicity - Envelope Composition

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2,2, figsize=(10,6), tight_layout='True')

ax0.plot(hmps, tot_pressure_real_1xsolar_melt100, color=color_tot, lw=3, ls= '-', label='Total')
ax0.plot(hmps, H2_pressure_real_1xsolar_melt100, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax0.plot(hmps, He_pressure_real_1xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
ax0.plot(hmps, H2O_pressure_real_1xsolar_melt100, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax0.plot(hmps, SiH4_pressure_real_1xsolar_melt100, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax0.plot(hmps, SiO_pressure_real_1xsolar_melt100, color=color_SiO, lw=2, ls= '-', label='SiO')
ax0.plot(hmps, CH4_pressure_real_1xsolar_melt100, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax0.plot(hmps, CO_pressure_real_1xsolar_melt100, color=color_CO, lw=2, ls= '-', label='CO')
ax0.plot(hmps, CO2_pressure_real_1xsolar_melt100, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax0.plot(hmps, N2_pressure_real_1xsolar_melt100, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax0.plot(hmps, NH3_pressure_real_1xsolar_melt100, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax0.plot(hmps, O2_pressure_real_1xsolar_melt100, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax0.set_title(r'(a) 1$\times$ solar, 100% melt')
ax0.set_ylim([2e-11, 2e5])
ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax0.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax0.set_xticks([0.1, 0.3, 1])
ax0.get_xaxis().set_major_formatter(ScalarFormatter())

ax1.plot(hmps, tot_pressure_real_100xsolar_melt100, color=color_tot, lw=3, ls= '-', label='Total')
ax1.plot(hmps, H2_pressure_real_100xsolar_melt100, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax1.plot(hmps, He_pressure_real_100xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
ax1.plot(hmps, H2O_pressure_real_100xsolar_melt100, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax1.plot(hmps, SiH4_pressure_real_100xsolar_melt100, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax1.plot(hmps, SiO_pressure_real_100xsolar_melt100, color=color_SiO, lw=2, ls= '-', label='SiO')
ax1.plot(hmps, CH4_pressure_real_100xsolar_melt100, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax1.plot(hmps, CO_pressure_real_100xsolar_melt100, color=color_CO, lw=2, ls= '-', label='CO')
ax1.plot(hmps, CO2_pressure_real_100xsolar_melt100, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax1.plot(hmps, N2_pressure_real_100xsolar_melt100, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax1.plot(hmps, NH3_pressure_real_100xsolar_melt100, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax1.plot(hmps, O2_pressure_real_100xsolar_melt100, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax1.set_title(r'(b) 100$\times$ solar, 100% melt')
ax1.set_ylim([2e-11, 2e5])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax1.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax1.set_xticks([0.1, 0.3, 1])
ax1.get_xaxis().set_major_formatter(ScalarFormatter())

ax2.plot(hmps, tot_pressure_real_1xsolar_melt1, color=color_tot, lw=3, ls= '-', label='Total')
ax2.plot(hmps, H2_pressure_real_1xsolar_melt1, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax2.plot(hmps, He_pressure_real_1xsolar_melt1, color=color_He, lw=2, ls= '-', label='He')
ax2.plot(hmps, H2O_pressure_real_1xsolar_melt1, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax2.plot(hmps, SiH4_pressure_real_1xsolar_melt1, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax2.plot(hmps, SiO_pressure_real_1xsolar_melt1, color=color_SiO, lw=2, ls= '-', label='SiO')
ax2.plot(hmps, CH4_pressure_real_1xsolar_melt1, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax2.plot(hmps, CO_pressure_real_1xsolar_melt1, color=color_CO, lw=2, ls= '-', label='CO')
ax2.plot(hmps, CO2_pressure_real_1xsolar_melt1, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax2.plot(hmps, N2_pressure_real_1xsolar_melt1, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax2.plot(hmps, NH3_pressure_real_1xsolar_melt1, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax2.plot(hmps, O2_pressure_real_1xsolar_melt1, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax2.set_title(r'(c) 1$\times$ solar, 1% melt') 
ax2.set_ylim([2e-11, 2e5])
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax2.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax2.set_xticks([0.1, 0.3, 1])
ax2.get_xaxis().set_major_formatter(ScalarFormatter())

ax3.plot(hmps, tot_pressure_real_100xsolar_melt1, color=color_tot, lw=3, ls= '-', label='Total')
ax3.plot(hmps, H2_pressure_real_100xsolar_melt1, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax3.plot(hmps, He_pressure_real_100xsolar_melt1, color=color_He, lw=2, ls= '-', label='He')
ax3.plot(hmps, H2O_pressure_real_100xsolar_melt1, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax3.plot(hmps, SiH4_pressure_real_100xsolar_melt1, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax3.plot(hmps, SiO_pressure_real_100xsolar_melt1, color=color_SiO, lw=2, ls= '-', label='SiO')
ax3.plot(hmps, CH4_pressure_real_100xsolar_melt1, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax3.plot(hmps, CO_pressure_real_100xsolar_melt1, color=color_CO, lw=2, ls= '-', label='CO')
ax3.plot(hmps, CO2_pressure_real_100xsolar_melt1, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax3.plot(hmps, N2_pressure_real_100xsolar_melt1, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax3.plot(hmps, NH3_pressure_real_100xsolar_melt1, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax3.plot(hmps, O2_pressure_real_100xsolar_melt1, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax3.set_title(r'(d) 100$\times$ solar, 1% melt') 
ax3.set_ylim([2e-11, 2e5])
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax3.set_ylabel(r'MEB Partial Pressure [bar]', fontsize=12)
ax3.set_xticks([0.1, 0.3, 1])
ax3.get_xaxis().set_major_formatter(ScalarFormatter())

ax00 = ax0.twinx()
ax00.set_yscale('log')
ax00.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax00.set_ylim([2e-15, 2e1])

ax11 = ax1.twinx()
ax11.set_yscale('log')
ax11.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax11.set_ylim([2e-15, 2e1])

ax22 = ax2.twinx()
ax22.set_yscale('log')
ax22.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax22.set_ylim([2e-15, 2e1])

ax33 = ax3.twinx()
ax33.set_yscale('log')
ax33.set_ylabel(r"MEB Partial Pressure [GPa]", fontsize=12)
ax33.set_ylim([2e-15, 2e1])

labelLines(ax0.get_lines())
labelLines(ax1.get_lines())
labelLines(ax2.get_lines())
labelLines(ax3.get_lines())

fig.suptitle(r'Envelope Composition at MEB (H$-$He$-$C$-$N$-$O$-$Si system)', fontsize=16)

plt.savefig("HHeCNOSi_envelope_fullrange.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_envelope_fullrange.png", bbox_inches='tight')
plt.show()

### Fig. B1 - Effects of Mantle Melt Fraction and Metallicity - Volatile Partitioning in Magma

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2,2, figsize=(9,5.5), tight_layout='True')

ax0.plot(hmps, 100 * (H_mass_tot_real_1xsolar_melt100-H_mass_atm_real_1xsolar_melt100)/H_mass_tot_real_1xsolar_melt100, color=color_H, lw=2, ls= '-', label='H')
ax0.plot(hmps, 100 * (He_mass_tot_real_1xsolar_melt100-He_mass_atm_real_1xsolar_melt100)/He_mass_tot_real_1xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
ax0.plot(hmps, 100 * (C_mass_tot_real_1xsolar_melt100-C_mass_atm_real_1xsolar_melt100)/C_mass_tot_real_1xsolar_melt100, color=color_C, lw=2, ls= '-', label='C')
ax0.plot(hmps, 100 * (N_mass_tot_real_1xsolar_melt100-N_mass_atm_real_1xsolar_melt100)/N_mass_tot_real_1xsolar_melt100, color=color_N, lw=2, ls= '-', label='N')

ax0.set_title(r'(a) 1$\times$ solar, 100% melt')

ax0.set_ylim([-5, 105])
ax0.set_xscale('log')
ax0.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax0.set_ylabel(r'Partitioning in Magma [%]', fontsize=12)
ax0.set_xticks([0.1, 0.3, 1])
ax0.get_xaxis().set_major_formatter(ScalarFormatter())

l1, = ax1.plot(hmps, 100 * (H_mass_tot_real_100xsolar_melt100-H_mass_atm_real_100xsolar_melt100)/H_mass_tot_real_100xsolar_melt100, color=color_H, lw=2, ls= '-', label='H')
l2, = ax1.plot(hmps, 100 * (He_mass_tot_real_100xsolar_melt100-He_mass_atm_real_100xsolar_melt100)/He_mass_tot_real_100xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')
l3, = ax1.plot(hmps, 100 * (C_mass_tot_real_100xsolar_melt100-C_mass_atm_real_100xsolar_melt100)/C_mass_tot_real_100xsolar_melt100, color=color_C, lw=2, ls= '-', label='C')
l4, = ax1.plot(hmps, 100 * (N_mass_tot_real_100xsolar_melt100-N_mass_atm_real_100xsolar_melt100)/N_mass_tot_real_100xsolar_melt100, color=color_N, lw=2, ls= '-', label='N')

ax1.set_title(r'(b) 100$\times$ solar, 100% melt')

ax1.set_ylim([-5, 105])
ax1.set_xscale('log')
ax1.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax1.set_ylabel(r'Partitioning in Magma [%]', fontsize=12)
ax1.set_xticks([0.1, 0.3, 1])
ax1.get_xaxis().set_major_formatter(ScalarFormatter())

ax2.plot(hmps, 100 * (H_mass_tot_real_1xsolar_melt1-H_mass_atm_real_1xsolar_melt1)/H_mass_tot_real_1xsolar_melt1, color=color_H, lw=2, ls= '-', label='H')
ax2.plot(hmps, 100 * (He_mass_tot_real_1xsolar_melt1-He_mass_atm_real_1xsolar_melt1)/He_mass_tot_real_1xsolar_melt1, color=color_He, lw=2, ls= '-', label='He')
ax2.plot(hmps, 100 * (C_mass_tot_real_1xsolar_melt1-C_mass_atm_real_1xsolar_melt1)/C_mass_tot_real_1xsolar_melt1, color=color_C, lw=2, ls= '-', label='C')
ax2.plot(hmps, 100 * (N_mass_tot_real_1xsolar_melt1-N_mass_atm_real_1xsolar_melt1)/N_mass_tot_real_1xsolar_melt1, color=color_N, lw=2, ls= '-', label='N')

ax2.set_title(r'(c) 1$\times$ solar, 1% melt')

ax2.set_ylim([-5, 105])
ax2.set_xscale('log')
ax2.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax2.set_ylabel(r'Partitioning in Magma [%]', fontsize=12)
ax2.set_xticks([0.1, 0.3, 1])
ax2.get_xaxis().set_major_formatter(ScalarFormatter())

ax3.plot(hmps, 100 * (H_mass_tot_real_100xsolar_melt1-H_mass_atm_real_100xsolar_melt1)/H_mass_tot_real_100xsolar_melt1, color=color_H, lw=2, ls= '-', label='H')
ax3.plot(hmps, 100 * (He_mass_tot_real_100xsolar_melt1-He_mass_atm_real_100xsolar_melt1)/He_mass_tot_real_100xsolar_melt1, color=color_He, lw=2, ls= '-', label='He')
ax3.plot(hmps, 100 * (C_mass_tot_real_100xsolar_melt1-C_mass_atm_real_100xsolar_melt1)/C_mass_tot_real_100xsolar_melt1, color=color_C, lw=2, ls= '-', label='C')
ax3.plot(hmps, 100 * (N_mass_tot_real_100xsolar_melt1-N_mass_atm_real_100xsolar_melt1)/N_mass_tot_real_100xsolar_melt1, color=color_N, lw=2, ls= '-', label='N')

ax3.set_title(r'(d) 100$\times$ solar, 1% melt')

ax3.set_ylim([-5, 105])
ax3.set_xscale('log')
ax3.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax3.set_ylabel(r'Partitioning in Magma [%]', fontsize=12)
ax3.set_xticks([0.1, 0.3, 1])
ax3.get_xaxis().set_major_formatter(ScalarFormatter())

labelLines(ax0.get_lines())
labelLines(ax1.get_lines())
labelLines(ax2.get_lines())
labelLines(ax3.get_lines())


fig.suptitle(r'Volatile Solubility in Magma (H$-$He$-$C$-$N$-$O$-$Si system)', fontsize=16) 

plt.savefig("HHeCNOSi_solubility.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_solubility.png", bbox_inches='tight')
plt.show()

### Fig. B2 - Effects of Mantle Melt Fraction and Metallicity - Fugacity Coefficients

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2,2, figsize=(9,5.5), tight_layout='True')

ax0.plot(hmps, H2_fc_real_1xsolar_melt100, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax0.plot(hmps, H2O_fc_real_1xsolar_melt100, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax0.plot(hmps, O2_fc_real_1xsolar_melt100, color=color_O2, lw=2, ls= '-', label='O$_2$')
ax0.plot(hmps, SiH4_fc_real_1xsolar_melt100, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax0.plot(hmps, SiO_fc_real_1xsolar_melt100, color=color_SiO, lw=2, ls= '-', label='SiO')
ax0.plot(hmps, CO2_fc_real_1xsolar_melt100, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax0.plot(hmps, CO_fc_real_1xsolar_melt100, color=color_CO, lw=2, ls= '-', label='CO')
ax0.plot(hmps, CH4_fc_real_1xsolar_melt100, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax0.plot(hmps, N2_fc_real_1xsolar_melt100, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax0.plot(hmps, NH3_fc_real_1xsolar_melt100, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax0.plot(hmps, He_fc_real_1xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')

ax0.set_title(r'(a) 1$\times$ solar, 100% melt')

ax0.set_ylim([8e-2, 1e7])
ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax0.set_ylabel(r'Fugacity Coefficient', fontsize=12)
ax0.set_xticks([0.1, 0.3, 1])
ax0.get_xaxis().set_major_formatter(ScalarFormatter())

ax1.plot(hmps, H2_fc_real_100xsolar_melt100, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax1.plot(hmps, H2O_fc_real_100xsolar_melt100, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax1.plot(hmps, O2_fc_real_100xsolar_melt100, color=color_O2, lw=2, ls= '-', label='O$_2$')
ax1.plot(hmps, SiH4_fc_real_100xsolar_melt100, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax1.plot(hmps, SiO_fc_real_100xsolar_melt100, color=color_SiO, lw=2, ls= '-', label='SiO')
ax1.plot(hmps, CO2_fc_real_100xsolar_melt100, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax1.plot(hmps, CO_fc_real_100xsolar_melt100, color=color_CO, lw=2, ls= '-', label='CO')
ax1.plot(hmps, CH4_fc_real_100xsolar_melt100, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax1.plot(hmps, N2_fc_real_100xsolar_melt100, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax1.plot(hmps, NH3_fc_real_100xsolar_melt100, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax1.plot(hmps, He_fc_real_100xsolar_melt100, color=color_He, lw=2, ls= '-', label='He')


ax1.set_title(r'(b) 100$\times$ solar, 100% melt')

ax1.set_ylim([8e-2, 1e7])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax1.set_ylabel(r'Fugacity Coefficient', fontsize=12)
ax1.set_xticks([0.1, 0.3, 1])
ax1.get_xaxis().set_major_formatter(ScalarFormatter())

ax2.plot(hmps, H2_fc_real_1xsolar_melt1, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax2.plot(hmps, H2O_fc_real_1xsolar_melt1, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax2.plot(hmps, O2_fc_real_1xsolar_melt1, color=color_O2, lw=2, ls= '-', label='O$_2$')
ax2.plot(hmps, SiH4_fc_real_1xsolar_melt1, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax2.plot(hmps, SiO_fc_real_1xsolar_melt1, color=color_SiO, lw=2, ls= '-', label='SiO')
ax2.plot(hmps, CO2_fc_real_1xsolar_melt1, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax2.plot(hmps, CO_fc_real_1xsolar_melt1, color=color_CO, lw=2, ls= '-', label='CO')
ax2.plot(hmps, CH4_fc_real_1xsolar_melt1, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax2.plot(hmps, N2_fc_real_1xsolar_melt1, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax2.plot(hmps, NH3_fc_real_1xsolar_melt1, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax2.plot(hmps, He_fc_real_1xsolar_melt1, color=color_He, lw=2, ls= '-', label='He')

ax2.set_title(r'(c) 1$\times$ solar, 1% melt')

ax2.set_ylim([8e-2, 1e7])
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax2.set_ylabel(r'Fugacity Coefficient', fontsize=12)
ax2.set_xticks([0.1, 0.3, 1])
ax2.get_xaxis().set_major_formatter(ScalarFormatter())

ax3.plot(hmps, H2_fc_real_100xsolar_melt1, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax3.plot(hmps, H2O_fc_real_100xsolar_melt1, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax3.plot(hmps, O2_fc_real_100xsolar_melt1, color=color_O2, lw=2, ls= '-', label='O$_2$')
ax3.plot(hmps, SiH4_fc_real_100xsolar_melt1, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax3.plot(hmps, SiO_fc_real_100xsolar_melt1, color=color_SiO, lw=2, ls= '-', label='SiO')
ax3.plot(hmps, CO2_fc_real_100xsolar_melt1, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax3.plot(hmps, CO_fc_real_100xsolar_melt1, color=color_CO, lw=2, ls= '-', label='CO')
ax3.plot(hmps, CH4_fc_real_100xsolar_melt1, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax3.plot(hmps, N2_fc_real_100xsolar_melt1, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax3.plot(hmps, NH3_fc_real_100xsolar_melt1, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
ax3.plot(hmps, He_fc_real_100xsolar_melt1, color=color_He, lw=2, ls= '-', label='He')

ax3.set_title(r'(d) 100$\times$ solar, 1% melt')

ax3.set_ylim([8e-2, 1e7])
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlabel(r'Hydrogen Mass / Planet Mass [%]', fontsize=14)
ax3.set_ylabel(r'Fugacity Coefficient', fontsize=12)
ax3.set_xticks([0.1, 0.3, 1])
ax3.get_xaxis().set_major_formatter(ScalarFormatter())

labelLines(ax0.get_lines())
labelLines(ax1.get_lines())
labelLines(ax2.get_lines())
labelLines(ax3.get_lines())

# ax1.legend(ncol=5, labelspacing=0, borderpad=0, columnspacing=0.1, fontsize=9, framealpha=0.2)

fig.suptitle(r'Real Gas Fugacity Coefficients (H$-$He$-$C$-$N$-$O$-$Si system)', fontsize=16) 

plt.savefig("HHeCNOSi_fugcoeff.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_fugcoeff.png", bbox_inches='tight')
plt.show()

## C. H-He-C-N-O-Si Atmosphere Composition

### Retrieve Data

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 1

### 1x solar

init_metallicity = 1

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt100_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

### 10x solar

init_metallicity = 10

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt100_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

### 100x solar

init_metallicity = 100

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt100_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 0.1

### 1x solar

init_metallicity = 1

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt10_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

### 10x solar

init_metallicity = 10

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt10_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

### 100x solar

init_metallicity = 100

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt10_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

In [ ]:
mantle_melt_fraction = 0.01

### 1x solar

init_metallicity = 1

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt1_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

### 10x solar

init_metallicity = 10

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt100_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]


### 100x solar

init_metallicity = 100

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt1_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

In [ ]:
mantle_melt_fraction = 0

### 1x solar

init_metallicity = 1

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt0_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

### 10x solar

init_metallicity = 10

## 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt000_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt000_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]


### 100x solar

init_metallicity = 100


### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt0_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

### Plots vertical speciation vs SiO2 cond.

In [ ]:
fig, ((ax0, ax1), (ax2, ax3), (ax4, ax5)) = plt.subplots(3,2, figsize=(10,9), tight_layout='True')

ax00 = ax0.twiny()
ax00.set_xlim([2, 23.8])
#ax00.set_xscale('log')
ax00.set_xlabel(r'Mean Molecular Weight [amu]', fontsize=12) 

ax00.axvspan(2.3, 2.7, alpha=0.5, color='black', label="MMW (Free Chem, Davenport et al. 2025)")
ax00.axvspan(2.3, 6.3, alpha=0.2, color='black', label="MMW (Eq. Chem, Davenport et al. 2025)")

mmw_obs, labels = ax00.get_legend_handles_labels()

ax00.plot(mu_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color='black', lw=3, ls= '-', label='MMW')

ax11 = ax1.twiny()
ax11.set_xlim([2, 23.8])
#ax11.set_xscale('log')
ax11.set_xlabel(r'Mean Molecular Weight [amu]', fontsize=12) 
ax11.plot(mu_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color='black', lw=3, ls= '-', label='MMW')
ax11.axvspan(2.3, 2.7, alpha=0.5, color='black', label="MMW (Free Chem, Davenport et al. 2025)")
ax11.axvspan(2.3, 6.3, alpha=0.2, color='black', label="MMW (Eq. Chem, Davenport et al. 2025)")

ax22 = ax2.twiny()
ax22.set_xlim([2, 23.8])
#ax22.set_xscale('log')
ax22.set_xlabel(r'Mean Molecular Weight [amu]', fontsize=12) 
ax22.plot(mu_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color='black', lw=3, ls= '-', label='MMW')
ax22.axvspan(2.3, 2.7, alpha=0.5, color='black', label="MMW (Free Chem, Davenport et al. 2025)")
ax22.axvspan(2.3, 6.3, alpha=0.2, color='black', label="MMW (Eq. Chem, Davenport et al. 2025)")

ax33 = ax3.twiny()
ax33.set_xlim([2, 23.8])
#ax33.set_xscale('log')
ax33.set_xlabel(r'Mean Molecular Weight [amu]', fontsize=12) 
ax33.plot(mu_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color='black', lw=3, ls= '-', label='MMW')
ax33.axvspan(2.3, 2.7, alpha=0.5, color='black', label="MMW (Free Chem, Davenport et al. 2025)")
ax33.axvspan(2.3, 6.3, alpha=0.2, color='black', label="MMW (Eq. Chem, Davenport et al. 2025)")

ax44 = ax4.twiny()
ax44.set_xlim([2, 23.8])
#ax44.set_xscale('log')
ax44.set_xlabel(r'Mean Molecular Weight [amu]', fontsize=12) 
ax44.plot(mu_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color='black', lw=3, ls= '-', label='MMW')
ax44.axvspan(2.3, 2.7, alpha=0.5, color='black', label="MMW (Free Chem, Davenport et al. 2025)")
ax44.axvspan(2.3, 6.3, alpha=0.2, color='black', label="MMW (Eq. Chem, Davenport et al. 2025)")

ax55 = ax5.twiny()
ax55.set_xlim([2, 23.8])
#ax55.set_xscale('log')
ax55.set_xlabel(r'Mean Molecular Weight [amu]', fontsize=12) 
ax55.plot(mu_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color='black', lw=3, ls= '-', label='MMW')
ax55.axvspan(2.3, 2.7, alpha=0.5, color='black', label="MMW (Free Chem, Davenport et al. 2025)")
ax55.axvspan(2.3, 6.3, alpha=0.2, color='black', label="MMW (Eq. Chem, Davenport et al. 2025)")

ax0.axvspan(10**-4.95, 10**-2.64, alpha=0.2, color=color_H2O, label='H$_2$O (Davenport et al. 2025)')
ax0.axvspan(10**-7.93, 10**-2.42, alpha=0.2, color=color_CO, label='CO (Davenport et al. 2025)')

gases_obs, labels = ax0.get_legend_handles_labels()

ax0.plot(H2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax0.plot(He_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_He, lw=2, ls= '-', label='He')
ax0.plot(H2O_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax0.plot(SiH4_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax0.plot(SiO_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_SiO, lw=2, ls= '-', label='SiO')
ax0.plot(CH4_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax0.plot(CO_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_CO, lw=2, ls= '-', label='CO')
ax0.plot(CO2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax0.plot(N2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax0.plot(NH3_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
#ax0.plot(O2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz, tot_pressure_1xsolar_melt100_1wtH_bqz, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax0.set_title(r'(a) 1 wt% H, 1$\times$ solar, 100% melt')
ax0.set_xlim([1e-10, 2e0])
ax0.set_ylim([1e2, 1e-3])
ax0.set_xscale('log')
ax0.set_yscale('log')
ax0.set_xlabel(r'Mixing Ratio', fontsize=14)
ax0.set_ylabel(r'Pressure [bar]', fontsize=14)

ax1.axvspan(10**-4.95, 10**-2.64, alpha=0.2, color=color_H2O)
ax1.axvspan(10**-7.93, 10**-2.42, alpha=0.2, color=color_CO)

ax1.plot(H2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax1.plot(He_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_He, lw=2, ls= '-', label='He')
ax1.plot(H2O_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax1.plot(SiH4_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax1.plot(SiO_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_SiO, lw=2, ls= '-', label='SiO')
ax1.plot(CH4_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax1.plot(CO_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_CO, lw=2, ls= '-', label='CO')
ax1.plot(CO2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax1.plot(N2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax1.plot(NH3_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
#ax1.plot(O2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz, tot_pressure_100xsolar_melt100_1wtH_bqz, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax1.set_title(r'(b) 1 wt% H, 100$\times$ solar, 100% melt')
ax1.set_xlim([1e-10, 2e0])
ax1.set_ylim([1e2, 1e-3])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Mixing Ratio', fontsize=14)
ax1.set_ylabel(r'Pressure [bar]', fontsize=14)


ax2.axvspan(10**-4.95, 10**-2.64, alpha=0.2, color=color_H2O, label='H$_2$O (Davenport et al. 2025)')
ax2.axvspan(10**-7.93, 10**-2.42, alpha=0.2, color=color_CO, label='CO (Davenport et al. 2025)')

ax2.plot(H2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax2.plot(He_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_He, lw=2, ls= '-', label='He')
ax2.plot(H2O_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax2.plot(SiH4_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax2.plot(SiO_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_SiO, lw=2, ls= '-', label='SiO')
ax2.plot(CH4_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax2.plot(CO_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_CO, lw=2, ls= '-', label='CO')
ax2.plot(CO2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax2.plot(N2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax2.plot(NH3_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
#ax2.plot(O2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz, tot_pressure_1xsolar_melt1_1wtH_bqz, color=color_O2, lw=2, ls= '-', label='O$_2$')


ax2.set_title(r'(c) 1 wt% H, 1$\times$ solar, 1% melt')
ax2.set_xlim([1e-10, 2e0])
ax2.set_ylim([1e2, 1e-3])
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel(r'Mixing Ratio', fontsize=14)
ax2.set_ylabel(r'Pressure [bar]', fontsize=14)

ax3.axvspan(10**-4.95, 10**-2.64, alpha=0.2, color=color_H2O, label='H$_2$O (Davenport et al. 2025)')
ax3.axvspan(10**-7.93, 10**-2.42, alpha=0.2, color=color_CO, label='CO (Davenport et al. 2025)')

ax3.plot(H2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax3.plot(He_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_He, lw=2, ls= '-', label='He')
ax3.plot(H2O_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax3.plot(SiH4_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax3.plot(SiO_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_SiO, lw=2, ls= '-', label='SiO')
ax3.plot(CH4_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax3.plot(CO_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_CO, lw=2, ls= '-', label='CO')
ax3.plot(CO2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax3.plot(N2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax3.plot(NH3_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
#ax3.plot(O2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz, tot_pressure_100xsolar_melt1_1wtH_bqz, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax3.set_title(r'(d) 1 wt% H, 100$\times$ solar, 1% melt')
ax3.set_xlim([1e-10, 2e0])
ax3.set_ylim([1e2, 1e-3])
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlabel(r'Mixing Ratio', fontsize=14)
ax3.set_ylabel(r'Pressure [bar]', fontsize=14)

ax4.axvspan(10**-4.95, 10**-2.64, alpha=0.2, color=color_H2O, label='H$_2$O (Davenport et al. 2025)')
ax4.axvspan(10**-7.93, 10**-2.42, alpha=0.2, color=color_CO, label='CO (Davenport et al. 2025)')

ax4.plot(H2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax4.plot(He_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_He, lw=2, ls= '-', label='He')
ax4.plot(H2O_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax4.plot(SiH4_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax4.plot(SiO_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_SiO, lw=2, ls= '-', label='SiO')
ax4.plot(CH4_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax4.plot(CO_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_CO, lw=2, ls= '-', label='CO')
ax4.plot(CO2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax4.plot(N2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax4.plot(NH3_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
#ax4.plot(O2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz, tot_pressure_1xsolar_melt0_1wtH_bqz, color=color_O2, lw=2, ls= '-', label='O$_2$')


ax4.set_title(r'(e) 1 wt% H, 1$\times$ solar, 0% melt')
ax4.set_xlim([1e-10, 2e0])
ax4.set_ylim([1e2, 1e-3])
ax4.set_xscale('log')
ax4.set_yscale('log')
ax4.set_xlabel(r'Mixing Ratio', fontsize=14)
ax4.set_ylabel(r'Pressure [bar]', fontsize=14)

ax5.axvspan(10**-4.95, 10**-2.64, alpha=0.2, color=color_H2O, label='H$_2$O (Davenport et al. 2025)')
ax5.axvspan(10**-7.93, 10**-2.42, alpha=0.2, color=color_CO, label='CO (Davenport et al. 2025)')

ax5.plot(H2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_H2, lw=2, ls= '-', label='H$_2$')
ax5.plot(He_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_He, lw=2, ls= '-', label='He')
ax5.plot(H2O_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_H2O, lw=2, ls= '-', label='H$_2$O')
ax5.plot(SiH4_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_SiH4, lw=2, ls= '-', label='SiH$_4$')
ax5.plot(SiO_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_SiO, lw=2, ls= '-', label='SiO')
ax5.plot(CH4_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_CH4, lw=2, ls= '-', label='CH$_4$')
ax5.plot(CO_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_CO, lw=2, ls= '-', label='CO')
ax5.plot(CO2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_CO2, lw=2, ls= '-', label='CO$_2$')
ax5.plot(N2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_N2, lw=2, ls= '-', label='N$_2$')
ax5.plot(NH3_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_NH3, lw=2, ls= '-', label='NH$_3$')
#ax5.plot(O2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz, tot_pressure_100xsolar_melt0_1wtH_bqz, color=color_O2, lw=2, ls= '-', label='O$_2$')

ax5.set_title(r'(f) 1 wt% H, 100$\times$ solar, 0% melt')
ax5.set_xlim([1e-10, 2e0])
ax5.set_ylim([1e2, 1e-3])
ax5.set_xscale('log')
ax5.set_yscale('log')
ax5.set_xlabel(r'Mixing Ratio', fontsize=14)
ax5.set_ylabel(r'Pressure [bar]', fontsize=14)

labelLines(ax0.get_lines(), align=False, yoffsets=[0.002, 0.02, 0.2, 2, 1, 0.008, 0, 0.8, 0.1, 30], xvals=[0.002, 0.02, 0.2, 0.6, 1e-7, 0.008, 1e-8, 0.8, 2, 1e-7])
labelLines(ax1.get_lines(), align=False, yoffsets=[0.002, 0.02, 0.2, 2, 1, 0.008,  8e-3, 8e-3, 2, 1e-1], xvals=[0.002, 0.02, 0.2, 2, 1e-7, 0.008, 1e-9, 1e-9, 2, 1e-7])
labelLines(ax2.get_lines(), align=False, yoffsets=[0.002, 0.02, 0.2, 2, 1, 0.008, 0, 0.8, 1, 1e-1], xvals=[0.002, 0.02, 0.2, 2, 1e-9, 0.008, 1e-8, 0.8, 2, 1e-6])
labelLines(ax3.get_lines(), align=False, yoffsets=[0.002, 0.2, 20, 0, 0, 0.008, 0.08, 0.002, 1, 0.002], xvals=[0.002, 0.2, 1e-1, 0, 0, 0.008, 0.08, 0.002, 1, 0.002])
labelLines(ax4.get_lines(), align=False, yoffsets=[0.002, 0.02, 20, 2, 20, 2, 0.008, 0.1, 1, 0.002], xvals=[0.002, 0.2, 1e-1, 0, 0, 0.008, 0.08, 0.002, 1, 0.002])
labelLines(ax5.get_lines(), align=False, yoffsets=[1, 20, 0.007, 1e1, 0, 2e1, 0.2, 0.002, 0.35, 1e-1], xvals=[0.7, 0.136, 0.005, 5e-10, 0, 1e-7, 0.04, 0.002, 0.0055, 1e-6])

labelLines(ax00.get_lines(), yoffsets=0.01)
labelLines(ax11.get_lines(), yoffsets=1)
labelLines(ax22.get_lines(), yoffsets=0.01)
labelLines(ax33.get_lines(), yoffsets=0.01)
labelLines(ax44.get_lines(), yoffsets=0.01)
labelLines(ax55.get_lines(), yoffsets=0.075, xvals=4.35)

fig.legend(
    handles = mmw_obs  + gases_obs,
    ncol=2,
    bbox_to_anchor=(0.5, -0.07), 
    labelspacing=0,
    borderpad=0,
    columnspacing=0.1,
    fontsize=12,
    loc="lower center"  
)
fig.suptitle(r'Equilibrium Atmospheric Composition with SiO$_2$ ($\beta$-quartz) Condensation', fontsize=16) 

plt.savefig("HHeCNOSi_atmosphere.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_atmosphere.png", bbox_inches='tight')
plt.show()

## D. SiH4/CH4 and Si/C Ratios at 10 mbar

### Retrieve Data

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 1

### 1x solar

init_metallicity = 1

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt100_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_1xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt100_1wtH_bqz,
                                            Si_moles_atm_1xsolar_melt100_1wtH_bqz/C_moles_atm_1xsolar_melt100_1wtH_bqz)
funcSiH4CH4_1xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt100_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz/CH4_volume_mixing_ratio_1xsolar_melt100_1wtH_bqz)

### 3x solar

init_metallicity = 3

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_3xsolar_melt100_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_3xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_3xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt100_1wtH_bqz,
                                            Si_moles_atm_3xsolar_melt100_1wtH_bqz/C_moles_atm_3xsolar_melt100_1wtH_bqz)
funcSiH4CH4_3xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt100_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz/CH4_volume_mixing_ratio_3xsolar_melt100_1wtH_bqz)

### 10x solar

init_metallicity = 10

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt100_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_10xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt100_1wtH_bqz,
                                            Si_moles_atm_10xsolar_melt100_1wtH_bqz/C_moles_atm_10xsolar_melt100_1wtH_bqz)
funcSiH4CH4_10xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt100_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz/CH4_volume_mixing_ratio_10xsolar_melt100_1wtH_bqz)

### 30x solar

init_metallicity = 30

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_30xsolar_melt100_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_30xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_30xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt100_1wtH_bqz,
                                            Si_moles_atm_30xsolar_melt100_1wtH_bqz/C_moles_atm_30xsolar_melt100_1wtH_bqz)
funcSiH4CH4_30xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt100_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz/CH4_volume_mixing_ratio_30xsolar_melt100_1wtH_bqz)

### 100x solar

init_metallicity = 100

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt100_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt100_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_100xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt100_1wtH_bqz,
                                            Si_moles_atm_100xsolar_melt100_1wtH_bqz/C_moles_atm_100xsolar_melt100_1wtH_bqz)
funcSiH4CH4_100xsolar_melt100_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt100_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz/CH4_volume_mixing_ratio_100xsolar_melt100_1wtH_bqz)

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 0.3

### 1x solar

init_metallicity = 1

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt30_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_1xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt30_1wtH_bqz,
                                            Si_moles_atm_1xsolar_melt30_1wtH_bqz/C_moles_atm_1xsolar_melt30_1wtH_bqz)
funcSiH4CH4_1xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt30_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz/CH4_volume_mixing_ratio_1xsolar_melt30_1wtH_bqz)

### 3x solar

init_metallicity = 3

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_3xsolar_melt30_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_3xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_3xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt30_1wtH_bqz,
                                            Si_moles_atm_3xsolar_melt30_1wtH_bqz/C_moles_atm_3xsolar_melt30_1wtH_bqz)
funcSiH4CH4_3xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt30_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz/CH4_volume_mixing_ratio_3xsolar_melt30_1wtH_bqz)

### 10x solar

init_metallicity = 10

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt30_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_10xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt30_1wtH_bqz,
                                            Si_moles_atm_10xsolar_melt30_1wtH_bqz/C_moles_atm_10xsolar_melt30_1wtH_bqz)
funcSiH4CH4_10xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt30_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz/CH4_volume_mixing_ratio_10xsolar_melt30_1wtH_bqz)

### 30x solar

init_metallicity = 30

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_30xsolar_melt30_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_30xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_30xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt30_1wtH_bqz,
                                            Si_moles_atm_30xsolar_melt30_1wtH_bqz/C_moles_atm_30xsolar_melt30_1wtH_bqz)
funcSiH4CH4_30xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt30_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz/CH4_volume_mixing_ratio_30xsolar_melt30_1wtH_bqz)

### 100x solar

init_metallicity = 100

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt30_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt30_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_100xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt30_1wtH_bqz,
                                            Si_moles_atm_100xsolar_melt30_1wtH_bqz/C_moles_atm_100xsolar_melt30_1wtH_bqz)
funcSiH4CH4_100xsolar_melt30_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt30_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz/CH4_volume_mixing_ratio_100xsolar_melt30_1wtH_bqz)

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 0.1

### 1x solar

init_metallicity = 1

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt10_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_1xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt10_1wtH_bqz,
                                            Si_moles_atm_1xsolar_melt10_1wtH_bqz/C_moles_atm_1xsolar_melt10_1wtH_bqz)
funcSiH4CH4_1xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt10_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz/CH4_volume_mixing_ratio_1xsolar_melt10_1wtH_bqz)

### 3x solar

init_metallicity = 3

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_3xsolar_melt10_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_3xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_3xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt10_1wtH_bqz,
                                            Si_moles_atm_3xsolar_melt10_1wtH_bqz/C_moles_atm_3xsolar_melt10_1wtH_bqz)
funcSiH4CH4_3xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt10_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz/CH4_volume_mixing_ratio_3xsolar_melt10_1wtH_bqz)

### 10x solar

init_metallicity = 10

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt10_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_10xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt10_1wtH_bqz,
                                            Si_moles_atm_10xsolar_melt10_1wtH_bqz/C_moles_atm_10xsolar_melt10_1wtH_bqz)
funcSiH4CH4_10xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt10_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz/CH4_volume_mixing_ratio_10xsolar_melt10_1wtH_bqz)

### 30x solar

init_metallicity = 30

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_30xsolar_melt10_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_30xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_30xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt10_1wtH_bqz,
                                            Si_moles_atm_30xsolar_melt10_1wtH_bqz/C_moles_atm_30xsolar_melt10_1wtH_bqz)
funcSiH4CH4_30xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt10_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz/CH4_volume_mixing_ratio_30xsolar_melt10_1wtH_bqz)

### 100x solar

init_metallicity = 100


### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt10_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt10_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_100xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt10_1wtH_bqz,
                                            Si_moles_atm_100xsolar_melt10_1wtH_bqz/C_moles_atm_100xsolar_melt10_1wtH_bqz)
funcSiH4CH4_100xsolar_melt10_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt10_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz/CH4_volume_mixing_ratio_100xsolar_melt10_1wtH_bqz)

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 0.03

### 1x solar

init_metallicity = 1

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt3_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_1xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt3_1wtH_bqz,
                                            Si_moles_atm_1xsolar_melt3_1wtH_bqz/C_moles_atm_1xsolar_melt3_1wtH_bqz)
funcSiH4CH4_1xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt3_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz/CH4_volume_mixing_ratio_1xsolar_melt3_1wtH_bqz)

### 3x solar

init_metallicity = 3

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_3xsolar_melt3_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_3xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_3xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt3_1wtH_bqz,
                                            Si_moles_atm_3xsolar_melt3_1wtH_bqz/C_moles_atm_3xsolar_melt3_1wtH_bqz)
funcSiH4CH4_3xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt3_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz/CH4_volume_mixing_ratio_3xsolar_melt3_1wtH_bqz)

### 10x solar

init_metallicity = 10

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt3_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_10xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt3_1wtH_bqz,
                                            Si_moles_atm_10xsolar_melt3_1wtH_bqz/C_moles_atm_10xsolar_melt3_1wtH_bqz)
funcSiH4CH4_10xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt3_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz/CH4_volume_mixing_ratio_10xsolar_melt3_1wtH_bqz)

### 30x solar

init_metallicity = 30

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_30xsolar_melt3_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_30xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_30xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt3_1wtH_bqz,
                                            Si_moles_atm_30xsolar_melt3_1wtH_bqz/C_moles_atm_30xsolar_melt3_1wtH_bqz)
funcSiH4CH4_30xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt3_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz/CH4_volume_mixing_ratio_30xsolar_melt3_1wtH_bqz)

### 100x solar

init_metallicity = 100

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt3_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt3_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_100xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt3_1wtH_bqz,
                                            Si_moles_atm_100xsolar_melt3_1wtH_bqz/C_moles_atm_100xsolar_melt3_1wtH_bqz)
funcSiH4CH4_100xsolar_melt3_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt3_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz/CH4_volume_mixing_ratio_100xsolar_melt3_1wtH_bqz)

In [ ]:
mantle_melt_fraction = 0.01

### 1x solar

init_metallicity = 1

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt1_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_1xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt1_1wtH_bqz,
                                            Si_moles_atm_1xsolar_melt1_1wtH_bqz/C_moles_atm_1xsolar_melt1_1wtH_bqz)
funcSiH4CH4_1xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt1_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz/CH4_volume_mixing_ratio_1xsolar_melt1_1wtH_bqz)

### 3x solar

init_metallicity = 3

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_3xsolar_melt1_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_3xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_3xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt1_1wtH_bqz,
                                            Si_moles_atm_3xsolar_melt1_1wtH_bqz/C_moles_atm_3xsolar_melt1_1wtH_bqz)
funcSiH4CH4_3xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt1_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz/CH4_volume_mixing_ratio_3xsolar_melt1_1wtH_bqz)

### 10x solar

init_metallicity = 10

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt1_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_10xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt1_1wtH_bqz,
                                            Si_moles_atm_10xsolar_melt1_1wtH_bqz/C_moles_atm_10xsolar_melt1_1wtH_bqz)
funcSiH4CH4_10xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt1_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz/CH4_volume_mixing_ratio_10xsolar_melt1_1wtH_bqz)

### 30x solar

init_metallicity = 30

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_30xsolar_melt1_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_30xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_30xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt1_1wtH_bqz,
                                            Si_moles_atm_30xsolar_melt1_1wtH_bqz/C_moles_atm_30xsolar_melt1_1wtH_bqz)
funcSiH4CH4_30xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt1_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz/CH4_volume_mixing_ratio_30xsolar_melt1_1wtH_bqz)

### 100x solar

init_metallicity = 100

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt1_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt1_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_100xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt1_1wtH_bqz,
                                            Si_moles_atm_100xsolar_melt1_1wtH_bqz/C_moles_atm_100xsolar_melt1_1wtH_bqz)
funcSiH4CH4_100xsolar_melt1_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt1_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz/CH4_volume_mixing_ratio_100xsolar_melt1_1wtH_bqz)

In [ ]:
mantle_melt_fraction = 0

### 1x solar

init_metallicity = 1

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]

He_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_1xsolar_melt0_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_1xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_1xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt0_1wtH_bqz,
                                            Si_moles_atm_1xsolar_melt0_1wtH_bqz/C_moles_atm_1xsolar_melt0_1wtH_bqz)
funcSiH4CH4_1xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_1xsolar_melt0_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz/CH4_volume_mixing_ratio_1xsolar_melt0_1wtH_bqz)

### 3x solar

init_metallicity = 3

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_3xsolar_melt0_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_3xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_3xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt0_1wtH_bqz,
                                            Si_moles_atm_3xsolar_melt0_1wtH_bqz/C_moles_atm_3xsolar_melt0_1wtH_bqz)
funcSiH4CH4_3xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_3xsolar_melt0_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz/CH4_volume_mixing_ratio_3xsolar_melt0_1wtH_bqz)

### 10x solar

init_metallicity = 10

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_10xsolar_melt0_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_10xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_10xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt0_1wtH_bqz,
                                            Si_moles_atm_10xsolar_melt0_1wtH_bqz/C_moles_atm_10xsolar_melt0_1wtH_bqz)
funcSiH4CH4_10xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_10xsolar_melt0_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz/CH4_volume_mixing_ratio_10xsolar_melt0_1wtH_bqz)

### 30x solar

init_metallicity = 30

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_30xsolar_melt0_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_30xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_30xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt0_1wtH_bqz,
                                            Si_moles_atm_30xsolar_melt0_1wtH_bqz/C_moles_atm_30xsolar_melt0_1wtH_bqz)
funcSiH4CH4_30xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_30xsolar_melt0_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz/CH4_volume_mixing_ratio_30xsolar_melt0_1wtH_bqz)

### 100x solar

init_metallicity = 100

### 1 wt% H - magma_sol_real - cond SiO2 bqz

filename = f"HHeCNOSi_vertical_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}_1wtH_bqz.xlsx"

H2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2_g")["volume_mixing_ratio"]
H2O_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H2O_g")["volume_mixing_ratio"]
O2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="O2_g")["volume_mixing_ratio"]
SiH4_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H4Si_g")["volume_mixing_ratio"]
SiO_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="OSi_g")["volume_mixing_ratio"]
CO2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO2_g")["volume_mixing_ratio"]
CO_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CO_g")["volume_mixing_ratio"]
CH4_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="CH4_g")["volume_mixing_ratio"]
N2_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="N2_g")["volume_mixing_ratio"]
NH3_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="H3N_g")["volume_mixing_ratio"]
He_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="He_g")["volume_mixing_ratio"]
tot_pressure_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]
temperature_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="atmosphere")["temperature"]
mu_100xsolar_melt0_1wtH_bqz = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_100xsolar_melt0_1wtH_bqz = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

funcSiC_100xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt0_1wtH_bqz,
                                            Si_moles_atm_100xsolar_melt0_1wtH_bqz/C_moles_atm_100xsolar_melt0_1wtH_bqz)
funcSiH4CH4_100xsolar_melt0_1wtH_bqz = interp1d(tot_pressure_100xsolar_melt0_1wtH_bqz,
                                                SiH4_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz/CH4_volume_mixing_ratio_100xsolar_melt0_1wtH_bqz)

### Plot Si/C

In [ ]:
zpoints

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(6,4.5), tight_layout='True')

x = [100, 30, 10, 3, 1, 0] # mantle melt fraction in per cent
y = [1, 3, 10, 30, 100] # metallicity in x solar units

X, Y = np.meshgrid(x, y, indexing='ij')
xpoints = X.flatten()
ypoints = Y.flatten()

zpoints = np.array([funcSiC_1xsolar_melt100_1wtH_bqz(1e-2), 
                    funcSiC_3xsolar_melt100_1wtH_bqz(1e-2), 
                    funcSiC_10xsolar_melt100_1wtH_bqz(1e-2),
                    funcSiC_30xsolar_melt100_1wtH_bqz(1e-2),
                    funcSiC_100xsolar_melt100_1wtH_bqz(1e-2),
                    funcSiC_1xsolar_melt30_1wtH_bqz(1e-2), 
                    funcSiC_3xsolar_melt30_1wtH_bqz(1e-2), 
                    funcSiC_10xsolar_melt30_1wtH_bqz(1e-2),
                    funcSiC_30xsolar_melt30_1wtH_bqz(1e-2),
                    funcSiC_100xsolar_melt30_1wtH_bqz(1e-2),
                    funcSiC_1xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiC_3xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiC_10xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiC_30xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiC_100xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiC_1xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiC_3xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiC_10xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiC_30xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiC_100xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiC_1xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiC_3xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiC_10xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiC_30xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiC_100xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiC_1xsolar_melt0_1wtH_bqz(1e-2), 
                    funcSiC_3xsolar_melt0_1wtH_bqz(1e-2), 
                    funcSiC_10xsolar_melt0_1wtH_bqz(1e-2), 
                    funcSiC_30xsolar_melt0_1wtH_bqz(1e-2), 
                    funcSiC_100xsolar_melt0_1wtH_bqz(1e-2)])

sc = ax1.scatter(xpoints, ypoints, c=zpoints, cmap='viridis', norm=LogNorm(vmin=1e-1, vmax=1e2))
ax1.set_xscale('linear')
ax1.set_xlim(-0.2, 0.5)
ax1.spines['right'].set_visible(False)
ax1.yaxis.set_ticks_position('left')

# Divider for shared y-axis
divider = make_axes_locatable(ax1)
axLog = divider.append_axes("right", size=3.8, pad=0.1, sharey=ax1)

# Same scatter on log axis
axLog.scatter(xpoints, ypoints, c=zpoints, cmap='viridis', norm=LogNorm(vmin=1e-1, vmax=1e2))
axLog.set_xscale('log')
axLog.set_xlim(0.5, 110)
axLog.spines['left'].set_visible(False)
axLog.yaxis.set_ticks_position('right')
axLog.yaxis.set_visible(False)
cbar_ax = divider.append_axes("right", size="80%", pad=0.1)
cbar = plt.colorbar(sc, cax=cbar_ax)
ticks = [0.1, 1, 10, 100]
labels = ["<0.1", "1", "10", ">100"]
cbar.set_ticks(ticks)
cbar.set_ticklabels(labels)
cbar.set_label('Si/C (molar ratio)', fontsize=14)

ax1.set_xticks([0])
ax1.set_xticklabels(['0'])

plt.setp(axLog.get_xticklabels(), visible=True)
plt.setp(ax1.get_xticklabels(), visible=True)


axLog.plot([6, 0.6], [110, 9], c='black', lw=1)
axLog.text(1.2, 25, r"Si/C $\sim$ 0.2", rotation=45)

# axLog.annotate('',
#              xy=(0.5, 100), xytext=(1.5, 30),
#              arrowprops=dict(facecolor='black')
#             )

axLog.annotate('',
             xy=(8, 4), xytext=(2.5, 15),
             arrowprops=dict(facecolor='black')
            )

axLog.text(4, 10, "Si/C", rotation=45)

ax1.set_yscale('log')
axLog.set_xlabel(r'Mantle Melt [%]', fontsize=14)
ax1.set_ylabel(r'Metallicity [$\times$ solar]', fontsize=14)


fig.suptitle(r'(a) Si/C ratio at 10 mbar', fontsize=16) 

plt.savefig("HHeCNOSi_SiCratios.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_SiCratios.png", bbox_inches='tight')
plt.show()

### Plot SiH4/CH4 ratios

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(6,4.5), tight_layout='True')

x = [100, 30, 10, 3, 1, 0] # mantle melt fraction in per cent
y = [1, 3, 10, 30, 100] # metallicity in x solar units

X, Y = np.meshgrid(x, y, indexing='ij')
xpoints = X.flatten()
ypoints = Y.flatten()

zpoints = np.array([funcSiH4CH4_1xsolar_melt100_1wtH_bqz(1e-2), 
                    funcSiH4CH4_3xsolar_melt100_1wtH_bqz(1e-2), 
                    funcSiH4CH4_10xsolar_melt100_1wtH_bqz(1e-2),
                    funcSiH4CH4_30xsolar_melt100_1wtH_bqz(1e-2),
                    funcSiH4CH4_100xsolar_melt100_1wtH_bqz(1e-2),
                    funcSiH4CH4_1xsolar_melt30_1wtH_bqz(1e-2), 
                    funcSiH4CH4_3xsolar_melt30_1wtH_bqz(1e-2), 
                    funcSiH4CH4_10xsolar_melt30_1wtH_bqz(1e-2),
                    funcSiH4CH4_30xsolar_melt30_1wtH_bqz(1e-2),
                    funcSiH4CH4_100xsolar_melt30_1wtH_bqz(1e-2),
                    funcSiH4CH4_1xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiH4CH4_3xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiH4CH4_10xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiH4CH4_30xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiH4CH4_100xsolar_melt10_1wtH_bqz(1e-2),
                    funcSiH4CH4_1xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiH4CH4_3xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiH4CH4_10xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiH4CH4_30xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiH4CH4_100xsolar_melt3_1wtH_bqz(1e-2),
                    funcSiH4CH4_1xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiH4CH4_3xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiH4CH4_10xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiH4CH4_30xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiH4CH4_100xsolar_melt1_1wtH_bqz(1e-2), 
                    funcSiH4CH4_1xsolar_melt0_1wtH_bqz(1e-2), 
                    funcSiH4CH4_3xsolar_melt0_1wtH_bqz(1e-2), 
                    funcSiH4CH4_10xsolar_melt0_1wtH_bqz(1e-2), 
                    funcSiH4CH4_30xsolar_melt0_1wtH_bqz(1e-2), 
                    funcSiH4CH4_100xsolar_melt0_1wtH_bqz(1e-2)])

sc = ax1.scatter(xpoints, ypoints, c=zpoints, cmap='viridis', norm=LogNorm(vmin=1e-1, vmax=1e2))
ax1.set_xscale('linear')
ax1.set_xlim(-0.2, 0.5)
ax1.spines['right'].set_visible(False)
ax1.yaxis.set_ticks_position('left')

# Divider for shared y-axis
divider = make_axes_locatable(ax1)
axLog = divider.append_axes("right", size=3.8, pad=0.1, sharey=ax1)

# Same scatter on log axis
axLog.scatter(xpoints, ypoints, c=zpoints, cmap='viridis', norm=LogNorm(vmin=1e-1, vmax=1e2))
axLog.set_xscale('log')
axLog.set_xlim(0.5, 110)
axLog.spines['left'].set_visible(False)
axLog.yaxis.set_ticks_position('right')
axLog.yaxis.set_visible(False)
cbar_ax = divider.append_axes("right", size="80%", pad=0.1)
cbar = plt.colorbar(sc, cax=cbar_ax)
ticks = [0.1, 1, 10, 100]
labels = ["<0.1", "1", "10", ">100"]
cbar.set_ticks(ticks)
cbar.set_ticklabels(labels)
cbar.set_label(r'SiH$_4$/CH$_4$ (molar ratio)', fontsize=14)

ax1.set_xticks([0])
ax1.set_xticklabels(['0'])

plt.setp(axLog.get_xticklabels(), visible=True)
plt.setp(ax1.get_xticklabels(), visible=True)


axLog.plot([6, 0.6], [110, 9], c='black', lw=1)
axLog.text(1.2, 25, r"SiH$_4$/CH$_4$ $\sim$ 0.2", rotation=45)

# axLog.annotate('',
#              xy=(0.5, 100), xytext=(1.5, 30),
#              arrowprops=dict(facecolor='black')
#             )

axLog.annotate('',
             xy=(8, 4), xytext=(2.5, 15),
             arrowprops=dict(facecolor='black')
            )

axLog.text(4, 10, "SiH$_4$/CH$_4$", rotation=45)

ax1.set_yscale('log')
axLog.set_xlabel(r'Mantle Melt [%]', fontsize=14)
ax1.set_ylabel(r'Metallicity [$\times$ solar]', fontsize=14)


fig.suptitle(r'(b) SiH$_4$/CH$_4$ ratio at 10 mbar', fontsize=16) 

plt.savefig("HHeCNOSi_SiH4CH4ratios.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_SiH4CH4ratios.png", bbox_inches='tight')
plt.show()

## E. H-He-C-N-O-Si Envelope Ratios in units of Metallicity

### Retrieve Data

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 1

# 1x solar 
init_metallicity = 1 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_1xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_1xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 3x solar
init_metallicity = 3

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_3xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_3xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 10x solar
init_metallicity = 10 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_10xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_10xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 30x solar
init_metallicity = 30

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_30xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_30xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 100x solar
init_metallicity = 100

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_100xsolar_melt100 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_100xsolar_melt100 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 0.3

# 1x solar 
init_metallicity = 1 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_1xsolar_melt30 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_1xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 3x solar
init_metallicity = 3

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_3xsolar_melt30 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_3xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 10x solar
init_metallicity = 10 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_10xsolar_melt30 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_10xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 30x solar
init_metallicity = 30

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_30xsolar_melt30 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_30xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 100x solar
init_metallicity = 100

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_100xsolar_melt30 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_100xsolar_melt30 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 0.1

# 1x solar 
init_metallicity = 1 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_1xsolar_melt10 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_1xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 3x solar
init_metallicity = 3

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_3xsolar_melt10 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_3xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 10x solar
init_metallicity = 10 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_10xsolar_melt10 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_10xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 30x solar
init_metallicity = 30

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_30xsolar_melt10 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_30xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 100x solar
init_metallicity = 100

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_100xsolar_melt10 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_100xsolar_melt10 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 0.03

# 1x solar 
init_metallicity = 1 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_1xsolar_melt3 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_1xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 3x solar
init_metallicity = 3

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_3xsolar_melt3 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_3xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 10x solar
init_metallicity = 10 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_10xsolar_melt3 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_10xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 30x solar
init_metallicity = 30

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_30xsolar_melt3 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_30xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 100x solar
init_metallicity = 100

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_100xsolar_melt3 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_100xsolar_melt3 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 0.01

# 1x solar 
init_metallicity = 1 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_1xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_1xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 3x solar
init_metallicity = 3

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_3xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_3xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 10x solar
init_metallicity = 10 

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_10xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_10xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 30x solar
init_metallicity = 30

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_30xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_30xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 100x solar
init_metallicity = 100

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

atm_mass_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="atmosphere")["mass"] / planet_mass * 100
mu_real_100xsolar_melt1 = 1e3 * pd.read_excel(filename, sheet_name="atmosphere")["molar_mass"]

H_moles_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_H")["total_mass"]
He_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_He")["total_mass"]
O_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_O")["total_mass"]
C_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_C")["total_mass"]
N_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_N")["total_mass"]
Si_mass_tot_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="element_Si")["total_mass"]

H2_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_100xsolar_melt1 = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

### Plot

In [ ]:
Si_met_1xsolar_melt100_1wtH_bqz = Si_moles_atm_real_1xsolar_melt100[49] / H_moles_atm_real_1xsolar_melt100[49] / (10**(Si_logN - H_logN))
O_met_1xsolar_melt100_1wtH_bqz = O_moles_atm_real_1xsolar_melt100[49] / H_moles_atm_real_1xsolar_melt100[49] / (10**(O_logN - H_logN))
He_met_1xsolar_melt100_1wtH_bqz = He_moles_atm_real_1xsolar_melt100[49] / H_moles_atm_real_1xsolar_melt100[49] / (10**(He_logN - H_logN))
C_met_1xsolar_melt100_1wtH_bqz = C_moles_atm_real_1xsolar_melt100[49] / H_moles_atm_real_1xsolar_melt100[49] / (10**(C_logN - H_logN))
N_met_1xsolar_melt100_1wtH_bqz = N_moles_atm_real_1xsolar_melt100[49] / H_moles_atm_real_1xsolar_melt100[49] / (10**(N_logN - H_logN))

Si_met_10xsolar_melt100_1wtH_bqz = Si_moles_atm_real_10xsolar_melt100[49] / H_moles_atm_real_10xsolar_melt100[49] / (10**(Si_logN - H_logN))
O_met_10xsolar_melt100_1wtH_bqz = O_moles_atm_real_10xsolar_melt100[49] / H_moles_atm_real_10xsolar_melt100[49] / (10**(O_logN - H_logN))
He_met_10xsolar_melt100_1wtH_bqz = He_moles_atm_real_10xsolar_melt100[49] / H_moles_atm_real_10xsolar_melt100[49] / (10**(He_logN - H_logN))
C_met_10xsolar_melt100_1wtH_bqz = C_moles_atm_real_10xsolar_melt100[49] / H_moles_atm_real_10xsolar_melt100[49] / (10**(C_logN - H_logN))
N_met_10xsolar_melt100_1wtH_bqz = N_moles_atm_real_10xsolar_melt100[49] / H_moles_atm_real_10xsolar_melt100[49] / (10**(N_logN - H_logN))

Si_met_100xsolar_melt100_1wtH_bqz = Si_moles_atm_real_100xsolar_melt100[49] / H_moles_atm_real_100xsolar_melt100[49] / (10**(Si_logN - H_logN))
O_met_100xsolar_melt100_1wtH_bqz = O_moles_atm_real_100xsolar_melt100[49] / H_moles_atm_real_100xsolar_melt100[49] / (10**(O_logN - H_logN))
He_met_100xsolar_melt100_1wtH_bqz = He_moles_atm_real_100xsolar_melt100[49] / H_moles_atm_real_100xsolar_melt100[49] / (10**(He_logN - H_logN))
C_met_100xsolar_melt100_1wtH_bqz = C_moles_atm_real_100xsolar_melt100[49] / H_moles_atm_real_100xsolar_melt100[49] / (10**(C_logN - H_logN))
N_met_100xsolar_melt100_1wtH_bqz = N_moles_atm_real_100xsolar_melt100[49] / H_moles_atm_real_100xsolar_melt100[49] / (10**(N_logN - H_logN))

Si_met_1xsolar_melt30_1wtH_bqz = Si_moles_atm_real_1xsolar_melt30[49] / H_moles_atm_real_1xsolar_melt30[49] / (10**(Si_logN - H_logN))
O_met_1xsolar_melt30_1wtH_bqz = O_moles_atm_real_1xsolar_melt30[49] / H_moles_atm_real_1xsolar_melt30[49] / (10**(O_logN - H_logN))
He_met_1xsolar_melt30_1wtH_bqz = He_moles_atm_real_1xsolar_melt30[49] / H_moles_atm_real_1xsolar_melt30[49] / (10**(He_logN - H_logN))
C_met_1xsolar_melt30_1wtH_bqz = C_moles_atm_real_1xsolar_melt30[49] / H_moles_atm_real_1xsolar_melt30[49] / (10**(C_logN - H_logN))
N_met_1xsolar_melt30_1wtH_bqz = N_moles_atm_real_1xsolar_melt30[49] / H_moles_atm_real_1xsolar_melt30[49] / (10**(N_logN - H_logN))

Si_met_10xsolar_melt30_1wtH_bqz = Si_moles_atm_real_10xsolar_melt30[49] / H_moles_atm_real_10xsolar_melt30[49] / (10**(Si_logN - H_logN))
O_met_10xsolar_melt30_1wtH_bqz = O_moles_atm_real_10xsolar_melt30[49] / H_moles_atm_real_10xsolar_melt30[49] / (10**(O_logN - H_logN))
He_met_10xsolar_melt30_1wtH_bqz = He_moles_atm_real_10xsolar_melt30[49] / H_moles_atm_real_10xsolar_melt30[49] / (10**(He_logN - H_logN))
C_met_10xsolar_melt30_1wtH_bqz = C_moles_atm_real_10xsolar_melt30[49] / H_moles_atm_real_10xsolar_melt30[49] / (10**(C_logN - H_logN))
N_met_10xsolar_melt30_1wtH_bqz = N_moles_atm_real_10xsolar_melt30[49] / H_moles_atm_real_10xsolar_melt30[49] / (10**(N_logN - H_logN))

Si_met_100xsolar_melt30_1wtH_bqz = Si_moles_atm_real_100xsolar_melt30[49] / H_moles_atm_real_100xsolar_melt30[49] / (10**(Si_logN - H_logN))
O_met_100xsolar_melt30_1wtH_bqz = O_moles_atm_real_100xsolar_melt30[49] / H_moles_atm_real_100xsolar_melt30[49] / (10**(O_logN - H_logN))
He_met_100xsolar_melt30_1wtH_bqz = He_moles_atm_real_100xsolar_melt30[49] / H_moles_atm_real_100xsolar_melt30[49] / (10**(He_logN - H_logN))
C_met_100xsolar_melt30_1wtH_bqz = C_moles_atm_real_100xsolar_melt30[49] / H_moles_atm_real_100xsolar_melt30[49] / (10**(C_logN - H_logN))
N_met_100xsolar_melt30_1wtH_bqz = N_moles_atm_real_100xsolar_melt30[49] / H_moles_atm_real_100xsolar_melt30[49] / (10**(N_logN - H_logN))

Si_met_1xsolar_melt10_1wtH_bqz = Si_moles_atm_real_1xsolar_melt10[49] / H_moles_atm_real_1xsolar_melt10[49] / (10**(Si_logN - H_logN))
O_met_1xsolar_melt10_1wtH_bqz = O_moles_atm_real_1xsolar_melt10[49] / H_moles_atm_real_1xsolar_melt10[49] / (10**(O_logN - H_logN))
He_met_1xsolar_melt10_1wtH_bqz = He_moles_atm_real_1xsolar_melt10[49] / H_moles_atm_real_1xsolar_melt10[49] / (10**(He_logN - H_logN))
C_met_1xsolar_melt10_1wtH_bqz = C_moles_atm_real_1xsolar_melt10[49] / H_moles_atm_real_1xsolar_melt10[49] / (10**(C_logN - H_logN))
N_met_1xsolar_melt10_1wtH_bqz = N_moles_atm_real_1xsolar_melt10[49] / H_moles_atm_real_1xsolar_melt10[49] / (10**(N_logN - H_logN))

Si_met_10xsolar_melt10_1wtH_bqz = Si_moles_atm_real_10xsolar_melt10[49] / H_moles_atm_real_10xsolar_melt10[49] / (10**(Si_logN - H_logN))
O_met_10xsolar_melt10_1wtH_bqz = O_moles_atm_real_10xsolar_melt10[49] / H_moles_atm_real_10xsolar_melt10[49] / (10**(O_logN - H_logN))
He_met_10xsolar_melt10_1wtH_bqz = He_moles_atm_real_10xsolar_melt10[49] / H_moles_atm_real_10xsolar_melt10[49] / (10**(He_logN - H_logN))
C_met_10xsolar_melt10_1wtH_bqz = C_moles_atm_real_10xsolar_melt10[49] / H_moles_atm_real_10xsolar_melt10[49] / (10**(C_logN - H_logN))
N_met_10xsolar_melt10_1wtH_bqz = N_moles_atm_real_10xsolar_melt10[49] / H_moles_atm_real_10xsolar_melt10[49] / (10**(N_logN - H_logN))

Si_met_100xsolar_melt10_1wtH_bqz = Si_moles_atm_real_100xsolar_melt10[49] / H_moles_atm_real_100xsolar_melt10[49] / (10**(Si_logN - H_logN))
O_met_100xsolar_melt10_1wtH_bqz = O_moles_atm_real_100xsolar_melt10[49] / H_moles_atm_real_100xsolar_melt10[49] / (10**(O_logN - H_logN))
He_met_100xsolar_melt10_1wtH_bqz = He_moles_atm_real_100xsolar_melt10[49] / H_moles_atm_real_100xsolar_melt10[49] / (10**(He_logN - H_logN))
C_met_100xsolar_melt10_1wtH_bqz = C_moles_atm_real_100xsolar_melt10[49] / H_moles_atm_real_100xsolar_melt10[49] / (10**(C_logN - H_logN))
N_met_100xsolar_melt10_1wtH_bqz = N_moles_atm_real_100xsolar_melt10[49] / H_moles_atm_real_100xsolar_melt10[49] / (10**(N_logN - H_logN))

Si_met_1xsolar_melt3_1wtH_bqz = Si_moles_atm_real_1xsolar_melt3[49] / H_moles_atm_real_1xsolar_melt3[49] / (10**(Si_logN - H_logN))
O_met_1xsolar_melt3_1wtH_bqz = O_moles_atm_real_1xsolar_melt3[49] / H_moles_atm_real_1xsolar_melt3[49] / (10**(O_logN - H_logN))
He_met_1xsolar_melt3_1wtH_bqz = He_moles_atm_real_1xsolar_melt3[49] / H_moles_atm_real_1xsolar_melt3[49] / (10**(He_logN - H_logN))
C_met_1xsolar_melt3_1wtH_bqz = C_moles_atm_real_1xsolar_melt3[49] / H_moles_atm_real_1xsolar_melt3[49] / (10**(C_logN - H_logN))
N_met_1xsolar_melt3_1wtH_bqz = N_moles_atm_real_1xsolar_melt3[49] / H_moles_atm_real_1xsolar_melt3[49] / (10**(N_logN - H_logN))

Si_met_10xsolar_melt3_1wtH_bqz = Si_moles_atm_real_10xsolar_melt3[49] / H_moles_atm_real_10xsolar_melt3[49] / (10**(Si_logN - H_logN))
O_met_10xsolar_melt3_1wtH_bqz = O_moles_atm_real_10xsolar_melt3[49] / H_moles_atm_real_10xsolar_melt3[49] / (10**(O_logN - H_logN))
He_met_10xsolar_melt3_1wtH_bqz = He_moles_atm_real_10xsolar_melt3[49] / H_moles_atm_real_10xsolar_melt3[49] / (10**(He_logN - H_logN))
C_met_10xsolar_melt3_1wtH_bqz = C_moles_atm_real_10xsolar_melt3[49] / H_moles_atm_real_10xsolar_melt3[49] / (10**(C_logN - H_logN))
N_met_10xsolar_melt3_1wtH_bqz = N_moles_atm_real_10xsolar_melt3[49] / H_moles_atm_real_10xsolar_melt3[49] / (10**(N_logN - H_logN))

Si_met_100xsolar_melt3_1wtH_bqz = Si_moles_atm_real_100xsolar_melt3[49] / H_moles_atm_real_100xsolar_melt3[49] / (10**(Si_logN - H_logN))
O_met_100xsolar_melt3_1wtH_bqz = O_moles_atm_real_100xsolar_melt3[49] / H_moles_atm_real_100xsolar_melt3[49] / (10**(O_logN - H_logN))
He_met_100xsolar_melt3_1wtH_bqz = He_moles_atm_real_100xsolar_melt3[49] / H_moles_atm_real_100xsolar_melt3[49] / (10**(He_logN - H_logN))
C_met_100xsolar_melt3_1wtH_bqz = C_moles_atm_real_100xsolar_melt3[49] / H_moles_atm_real_100xsolar_melt3[49] / (10**(C_logN - H_logN))
N_met_100xsolar_melt3_1wtH_bqz = N_moles_atm_real_100xsolar_melt3[49] / H_moles_atm_real_100xsolar_melt3[49] / (10**(N_logN - H_logN))

Si_met_1xsolar_melt1_1wtH_bqz = Si_moles_atm_real_1xsolar_melt1[49] / H_moles_atm_real_1xsolar_melt1[49] / (10**(Si_logN - H_logN))
O_met_1xsolar_melt1_1wtH_bqz = O_moles_atm_real_1xsolar_melt1[49] / H_moles_atm_real_1xsolar_melt1[49] / (10**(O_logN - H_logN))
He_met_1xsolar_melt1_1wtH_bqz = He_moles_atm_real_1xsolar_melt1[49] / H_moles_atm_real_1xsolar_melt1[49] / (10**(He_logN - H_logN))
C_met_1xsolar_melt1_1wtH_bqz = C_moles_atm_real_1xsolar_melt1[49] / H_moles_atm_real_1xsolar_melt1[49] / (10**(C_logN - H_logN))
N_met_1xsolar_melt1_1wtH_bqz = N_moles_atm_real_1xsolar_melt1[49] / H_moles_atm_real_1xsolar_melt1[49] / (10**(N_logN - H_logN))

Si_met_10xsolar_melt1_1wtH_bqz = Si_moles_atm_real_10xsolar_melt1[49] / H_moles_atm_real_10xsolar_melt1[49] / (10**(Si_logN - H_logN))
O_met_10xsolar_melt1_1wtH_bqz = O_moles_atm_real_10xsolar_melt1[49] / H_moles_atm_real_10xsolar_melt1[49] / (10**(O_logN - H_logN))
He_met_10xsolar_melt1_1wtH_bqz = He_moles_atm_real_10xsolar_melt1[49] / H_moles_atm_real_10xsolar_melt1[49] / (10**(He_logN - H_logN))
C_met_10xsolar_melt1_1wtH_bqz = C_moles_atm_real_10xsolar_melt1[49] / H_moles_atm_real_10xsolar_melt1[49] / (10**(C_logN - H_logN))
N_met_10xsolar_melt1_1wtH_bqz = N_moles_atm_real_10xsolar_melt1[49] / H_moles_atm_real_10xsolar_melt1[49] / (10**(N_logN - H_logN))

Si_met_100xsolar_melt1_1wtH_bqz = Si_moles_atm_real_100xsolar_melt1[49] / H_moles_atm_real_100xsolar_melt1[49] / (10**(Si_logN - H_logN))
O_met_100xsolar_melt1_1wtH_bqz = O_moles_atm_real_100xsolar_melt1[49] / H_moles_atm_real_100xsolar_melt1[49] / (10**(O_logN - H_logN))
He_met_100xsolar_melt1_1wtH_bqz = He_moles_atm_real_100xsolar_melt1[49] / H_moles_atm_real_100xsolar_melt1[49] / (10**(He_logN - H_logN))
C_met_100xsolar_melt1_1wtH_bqz = C_moles_atm_real_100xsolar_melt1[49] / H_moles_atm_real_100xsolar_melt1[49] / (10**(C_logN - H_logN))
N_met_100xsolar_melt1_1wtH_bqz = N_moles_atm_real_100xsolar_melt1[49] / H_moles_atm_real_100xsolar_melt1[49] / (10**(N_logN - H_logN))

Si_met_magma_sol_ideal = Si_moles_atm_magma_sol_ideal / H_moles_atm_magma_sol_ideal / (10**(Si_logN - H_logN))
O_met_magma_sol_ideal = O_moles_atm_magma_sol_ideal / H_moles_atm_magma_sol_ideal / (10**(O_logN - H_logN))

Si_met_magma_sol_real = Si_moles_atm_magma_sol_real / H_moles_atm_magma_sol_real / (10**(Si_logN - H_logN))
O_met_magma_sol_real = O_moles_atm_magma_sol_real / H_moles_atm_magma_sol_real / (10**(O_logN - H_logN))

fig, ax1 = plt.subplots(1, figsize=(7,4), tight_layout='True')

ax1.axhline(y=1, color='black', ls='-')
ax1.axhline(y=10, color='black', ls='--')
ax1.axhline(y=100, color='black', ls=':')

plt.text(40, 1.5, r"1$\times$ solar")
plt.text(40, 15, r"10$\times$ solar")
plt.text(40, 150, r"100$\times$ solar")

ax1.scatter(100, Si_met_1xsolar_melt100_1wtH_bqz, color=color_Si, s=30, marker="o", label=r"Si/H")
ax1.scatter(100, O_met_1xsolar_melt100_1wtH_bqz, color=color_O, s=30, marker="o", label=r"O/H")
ax1.scatter(100, He_met_1xsolar_melt100_1wtH_bqz, color=color_He, s=30, marker="o", label=r"He/H")
ax1.scatter(100, C_met_1xsolar_melt100_1wtH_bqz, color=color_C, s=30, marker="o", label=r"C/H")
ax1.scatter(100, N_met_1xsolar_melt100_1wtH_bqz, color=color_N, s=30, marker="o", label=r"N/H")

ax1.scatter(100, Si_met_10xsolar_melt100_1wtH_bqz, color=color_Si, s=30, marker="s")
ax1.scatter(100, O_met_10xsolar_melt100_1wtH_bqz, color=color_O, s=30, marker="s")
ax1.scatter(100, He_met_10xsolar_melt100_1wtH_bqz, color=color_He, s=30, marker="s")
ax1.scatter(100, C_met_10xsolar_melt100_1wtH_bqz, color=color_C, s=30, marker="s")
ax1.scatter(100, N_met_10xsolar_melt100_1wtH_bqz, color=color_N, s=30, marker="s")

ax1.scatter(100, Si_met_100xsolar_melt100_1wtH_bqz, color=color_Si, s=30, marker="D")
ax1.scatter(100, O_met_100xsolar_melt100_1wtH_bqz, color=color_O, s=30, marker="D")
ax1.scatter(100, He_met_100xsolar_melt100_1wtH_bqz, color=color_He, s=30, marker="D")
ax1.scatter(100, C_met_100xsolar_melt100_1wtH_bqz, color=color_C, s=30, marker="D")
ax1.scatter(100, N_met_100xsolar_melt100_1wtH_bqz, color=color_N, s=30, marker="D")

ax1.scatter(30, Si_met_1xsolar_melt30_1wtH_bqz, color=color_Si, s=30, marker="o")
ax1.scatter(30, O_met_1xsolar_melt30_1wtH_bqz, color=color_O, s=30, marker="o")
ax1.scatter(30, He_met_1xsolar_melt30_1wtH_bqz, color=color_He, s=30, marker="o")
ax1.scatter(30, C_met_1xsolar_melt30_1wtH_bqz, color=color_C, s=30, marker="o")
ax1.scatter(30, N_met_1xsolar_melt30_1wtH_bqz, color=color_N, s=30, marker="o")

ax1.scatter(30, Si_met_10xsolar_melt30_1wtH_bqz, color=color_Si, s=30, marker="s")
ax1.scatter(30, O_met_10xsolar_melt30_1wtH_bqz, color=color_O, s=30, marker="s")
ax1.scatter(30, He_met_10xsolar_melt30_1wtH_bqz, color=color_He, s=30, marker="s")
ax1.scatter(30, C_met_10xsolar_melt30_1wtH_bqz, color=color_C, s=30, marker="s")
ax1.scatter(30, N_met_10xsolar_melt30_1wtH_bqz, color=color_N, s=30, marker="s")

ax1.scatter(30, Si_met_100xsolar_melt30_1wtH_bqz, color=color_Si, s=30, marker="D")
ax1.scatter(30, O_met_100xsolar_melt30_1wtH_bqz, color=color_O, s=30, marker="D")
ax1.scatter(30, He_met_100xsolar_melt30_1wtH_bqz, color=color_He, s=30, marker="D")
ax1.scatter(30, C_met_100xsolar_melt30_1wtH_bqz, color=color_C, s=30, marker="D")
ax1.scatter(30, N_met_100xsolar_melt30_1wtH_bqz, color=color_N, s=30, marker="D")

ax1.scatter(10, Si_met_1xsolar_melt10_1wtH_bqz, color=color_Si, s=30, marker="o")
ax1.scatter(10, O_met_1xsolar_melt10_1wtH_bqz, color=color_O, s=30, marker="o")
ax1.scatter(10, He_met_1xsolar_melt10_1wtH_bqz, color=color_He, s=30, marker="o")
ax1.scatter(10, C_met_1xsolar_melt10_1wtH_bqz, color=color_C, s=30, marker="o")
ax1.scatter(10, N_met_1xsolar_melt10_1wtH_bqz, color=color_N, s=30, marker="o")

ax1.scatter(10, Si_met_10xsolar_melt10_1wtH_bqz, color=color_Si, s=30, marker="s")
ax1.scatter(10, O_met_10xsolar_melt10_1wtH_bqz, color=color_O, s=30, marker="s")
ax1.scatter(10, He_met_10xsolar_melt10_1wtH_bqz, color=color_He, s=30, marker="s")
ax1.scatter(10, C_met_10xsolar_melt10_1wtH_bqz, color=color_C, s=30, marker="s")
ax1.scatter(10, N_met_10xsolar_melt10_1wtH_bqz, color=color_N, s=30, marker="s")

ax1.scatter(10, Si_met_100xsolar_melt10_1wtH_bqz, color=color_Si, s=30, marker="D")
ax1.scatter(10, O_met_100xsolar_melt10_1wtH_bqz, color=color_O, s=30, marker="D")
ax1.scatter(10, He_met_100xsolar_melt10_1wtH_bqz, color=color_He, s=30, marker="D")
ax1.scatter(10, C_met_100xsolar_melt10_1wtH_bqz, color=color_C, s=30, marker="D")
ax1.scatter(10, N_met_100xsolar_melt10_1wtH_bqz, color=color_N, s=30, marker="D")

ax1.scatter(3, Si_met_1xsolar_melt3_1wtH_bqz, color=color_Si, s=30, marker="o")
ax1.scatter(3, O_met_1xsolar_melt3_1wtH_bqz, color=color_O, s=30, marker="o")
ax1.scatter(3, He_met_1xsolar_melt3_1wtH_bqz, color=color_He, s=30, marker="o")
ax1.scatter(3, C_met_1xsolar_melt3_1wtH_bqz, color=color_C, s=30, marker="o")
ax1.scatter(3, N_met_1xsolar_melt3_1wtH_bqz, color=color_N, s=30, marker="o")

ax1.scatter(3, Si_met_10xsolar_melt3_1wtH_bqz, color=color_Si, s=30, marker="s")
ax1.scatter(3, O_met_10xsolar_melt3_1wtH_bqz, color=color_O, s=30, marker="s")
ax1.scatter(3, He_met_10xsolar_melt3_1wtH_bqz, color=color_He, s=30, marker="s")
ax1.scatter(3, C_met_10xsolar_melt3_1wtH_bqz, color=color_C, s=30, marker="s")
ax1.scatter(3, N_met_10xsolar_melt3_1wtH_bqz, color=color_N, s=30, marker="s")

ax1.scatter(3, Si_met_100xsolar_melt3_1wtH_bqz, color=color_Si, s=30, marker="D")
ax1.scatter(3, O_met_100xsolar_melt3_1wtH_bqz, color=color_O, s=30, marker="D")
ax1.scatter(3, He_met_100xsolar_melt3_1wtH_bqz, color=color_He, s=30, marker="D")
ax1.scatter(3, C_met_100xsolar_melt3_1wtH_bqz, color=color_C, s=30, marker="D")
ax1.scatter(3, N_met_100xsolar_melt3_1wtH_bqz, color=color_N, s=30, marker="D")

ax1.scatter(1, Si_met_1xsolar_melt1_1wtH_bqz, color=color_Si, s=30, marker="o")
ax1.scatter(1, O_met_1xsolar_melt1_1wtH_bqz, color=color_O, s=30, marker="o")
ax1.scatter(1, He_met_1xsolar_melt1_1wtH_bqz, color=color_He, s=30, marker="o")
ax1.scatter(1, C_met_1xsolar_melt1_1wtH_bqz, color=color_C, s=30, marker="o")
ax1.scatter(1, N_met_1xsolar_melt1_1wtH_bqz, color=color_N, s=30, marker="o")

ax1.scatter(1, Si_met_10xsolar_melt1_1wtH_bqz, color=color_Si, s=30, marker="s")
ax1.scatter(1, O_met_10xsolar_melt1_1wtH_bqz, color=color_O, s=30, marker="s")
ax1.scatter(1, He_met_10xsolar_melt1_1wtH_bqz, color=color_He, s=30, marker="s")
ax1.scatter(1, C_met_10xsolar_melt1_1wtH_bqz, color=color_C, s=30, marker="s")
ax1.scatter(1, N_met_10xsolar_melt1_1wtH_bqz, color=color_N, s=30, marker="s")

ax1.scatter(1, Si_met_100xsolar_melt1_1wtH_bqz, color=color_Si, s=30, marker="D")
ax1.scatter(1, O_met_100xsolar_melt1_1wtH_bqz, color=color_O, s=30, marker="D")
ax1.scatter(1, He_met_100xsolar_melt1_1wtH_bqz, color=color_He, s=30, marker="D")
ax1.scatter(1, C_met_100xsolar_melt1_1wtH_bqz, color=color_C, s=30, marker="D")
ax1.scatter(1, N_met_100xsolar_melt1_1wtH_bqz, color=color_N, s=30, marker="D")

ax1.set_title(r'Envelope Elemental Ratios under Magma$-$Envelope Coupling')
ax1.set_xlim([9e-1, 1.1e2])
ax1.set_ylim([1e-4, 1e5])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Mantle Melt [%]', fontsize=14)
ax1.set_ylabel(r'Elemental Ratios [$\times$ solar]', fontsize=14)


custom_handles = [Line2D([], [], color='black', marker="D", ls="None", label=r'100$\times$ solar'),
                  Line2D([], [], color='black', marker="s", ls="None", label=r'10$\times$ solar'),
                  Line2D([], [], color='black', marker="o", ls="None", label=r'1$\times$ solar'),
                  Line2D([0], [0], color=color_He, ls="-", lw=3, label=r'He/H'),
                  Line2D([0], [0], color=color_C, ls="-", lw=3,label=r'C/H'),
                  Line2D([0], [0], color=color_N, ls="-", lw=3,label=r'N/H'),
                  Line2D([0], [0], color=color_O, ls="-", lw=3,label=r'O/H'),
                  Line2D([0], [0], color=color_Si, ls="-", lw=3, label=r'Si/H'),
                  ]

ax1.text(1.2, 0.03, 'Solubility M > Solubility H\nFugacity > Partial Pressure')
ax1.annotate('', 
             xy=(1.02, 2e-3), xytext=(1.02, 2e-1),
            arrowprops=dict(facecolor='black'),
            )

ax1.text(1.2, 1e3, 'Magma Vapourisation\nSolubility M < Solubility H\nFugacity < Partial Pressure')
ax1.annotate('', 
             xy=(1.02, 1e5), xytext=(1.02, 1e3),
            arrowprops=dict(facecolor='black'),
            )

ax1.legend(handles= custom_handles, ncol=2,  labelspacing=0.2, borderpad=0.2, columnspacing=0.5, bbox_to_anchor=(0.5, -0.01), loc='lower center')

plt.savefig("HHeCNOSi_ratios.pdf", bbox_inches='tight')
plt.savefig("HHeCNOSi_ratios.png", bbox_inches='tight')
plt.show()

# Sanity Checks

## Verify Mass Balance for H-O-Si

## Verify Mass Balance for H-He-C-N-O-Si

In [ ]:
error_threshold = 1e-8

mantle_melt_fractions = [1, 0.3, 0.1, 0.03, 0.01] # mantle melt fraction
init_metallicitys = [1, 3, 10, 30, 100] # metallicity in x solar units

for mantle_melt_fraction in mantle_melt_fractions:
    for init_metallicity in init_metallicitys:

        print(f"Checking mass balance for mantle melt fraction: {mantle_melt_fraction} and initial metallicity: {init_metallicity}x solar")
        filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

        Si_total_mass = si_kg_magma * mantle_melt_fraction + si_kgs_solar * init_metallicity # pd.read_excel(filename, sheet_name="element_Si")["total_mass"].values # 
        Si_atmosphere_mass = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"].values
        Si_condensed_mass = pd.read_excel(filename, sheet_name="element_Si")["condensed_mass"].values
        Si_dissolved_mass = pd.read_excel(filename, sheet_name="element_Si")["dissolved_mass"].values
        Si_difference_mass = Si_total_mass - (Si_atmosphere_mass + Si_condensed_mass + Si_dissolved_mass)
        Si_error_fraction = Si_difference_mass / Si_total_mass

        O_total_mass = o_kg_magma * mantle_melt_fraction + o_kgs_solar * init_metallicity #pd.read_excel(filename, sheet_name="element_O")["total_mass"].values
        O_atmosphere_mass = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"].values
        O_condensed_mass = pd.read_excel(filename, sheet_name="element_O")["condensed_mass"].values
        O_dissolved_mass = pd.read_excel(filename, sheet_name="element_O")["dissolved_mass"].values
        O_difference_mass = O_total_mass - (O_atmosphere_mass + O_condensed_mass + O_dissolved_mass)
        O_error_fraction = O_difference_mass / O_total_mass

        H_total_mass = h_kgs # pd.read_excel(filename, sheet_name="element_H")["total_mass"].values
        H_atmosphere_mass = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"].values
        H_condensed_mass = pd.read_excel(filename, sheet_name="element_H")["condensed_mass"].values
        H_dissolved_mass = pd.read_excel(filename, sheet_name="element_H")["dissolved_mass"].values
        H_difference_mass = H_total_mass - (H_atmosphere_mass + H_condensed_mass + H_dissolved_mass)
        H_error_fraction = H_difference_mass / H_total_mass

        He_total_mass = he_kgs_solar #pd.read_excel(filename, sheet_name="element_He")["total_mass"].values
        He_atmosphere_mass = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"].values
        He_condensed_mass = pd.read_excel(filename, sheet_name="element_He")["condensed_mass"].values
        He_dissolved_mass = pd.read_excel(filename, sheet_name="element_He")["dissolved_mass"].values
        He_difference_mass = He_total_mass - (He_atmosphere_mass + He_condensed_mass + He_dissolved_mass)
        He_error_fraction = He_difference_mass / He_total_mass

        C_total_mass = c_kgs_solar * init_metallicity # pd.read_excel(filename, sheet_name="element_C")["total_mass"].values
        C_atmosphere_mass = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"].values
        C_condensed_mass = pd.read_excel(filename, sheet_name="element_C")["condensed_mass"].values
        C_dissolved_mass = pd.read_excel(filename, sheet_name="element_C")["dissolved_mass"].values
        C_difference_mass = C_total_mass - (C_atmosphere_mass + C_condensed_mass + C_dissolved_mass)
        C_error_fraction = C_difference_mass / C_total_mass

        N_total_mass = n_kgs_solar * init_metallicity # pd.read_excel(filename, sheet_name="element_N")["total_mass"].values
        N_atmosphere_mass = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"].values
        N_condensed_mass = pd.read_excel(filename, sheet_name="element_N")["condensed_mass"].values
        N_dissolved_mass = pd.read_excel(filename, sheet_name="element_N")["dissolved_mass"].values
        N_difference_mass = N_total_mass - (N_atmosphere_mass + N_condensed_mass + N_dissolved_mass)
        N_error_fraction = N_difference_mass / N_total_mass

        if any(abs(Si_error_fraction) > error_threshold) or any(abs(O_error_fraction) > error_threshold) or any(abs(H_error_fraction) > error_threshold) or any(abs(He_error_fraction) > error_threshold) or any(abs(C_error_fraction) > error_threshold) or any(abs(N_error_fraction) > error_threshold):
            print(f"Mass balance error exceeds {error_threshold} for one or more elements.")
        error_counter_Si = 0
        for i in range(len(Si_error_fraction)):
            if abs(Si_error_fraction[i]) > error_threshold:
                error_counter_Si = error_counter_Si + 1
        error_counter_O = 0
        for i in range(len(O_error_fraction)):
            if abs(O_error_fraction[i]) > error_threshold:
                error_counter_O = error_counter_O + 1
        error_counter_H = 0
        for i in range(len(H_error_fraction)):
            if abs(H_error_fraction[i]) > error_threshold:
                error_counter_H = error_counter_H + 1
        error_counter_C = 0
        for i in range(len(C_error_fraction)):
            if abs(C_error_fraction[i]) > error_threshold:
                error_counter_C = error_counter_C + 1
        error_counter_N = 0
        for i in range(len(N_error_fraction)):
            if abs(N_error_fraction[i]) > error_threshold:
                error_counter_N = error_counter_N + 1
        error_counter_He = 0
        for i in range(len(He_error_fraction)):
            if abs(He_error_fraction[i]) > error_threshold:
                error_counter_He = error_counter_He + 1
                print(i, He_error_fraction[i], He_total_mass[i], hmps[i])

        print(init_metallicity, "x solar", round(100 * mantle_melt_fraction), "% melt")
        print(error_counter_Si,error_counter_O,error_counter_H,error_counter_C,error_counter_N,error_counter_He)
        print(f"Si Mass Balance Error: {any(abs(Si_error_fraction) > error_threshold)}")
        print(f"O Mass Balance Error: {any(abs(O_error_fraction) > error_threshold)}")
        print(f"H Mass Balance Error: {any(abs(H_error_fraction) > error_threshold)}")
        print(f"He Mass Balance Error: {any(abs(He_error_fraction) > error_threshold)}")
        print(f"C Mass Balance Error: {any(abs(C_error_fraction) > error_threshold)}")
        print(f"N Mass Balance Error: {any(abs(N_error_fraction) > error_threshold)}")


## Total Moles

### Retrieve Data

In [ ]:
# Update mantle melt fraction below
mantle_melt_fraction = 1

# 1x solar
init_metallicity = 1

### real + solubility

filename = f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_1xsolar = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

H_moles_atm_real_1xsolar = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_1xsolar = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_1xsolar = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_1xsolar = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_1xsolar = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_1xsolar = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_moles_melt_real_1xsolar = pd.read_excel(filename, sheet_name="element_H")["dissolved_moles"]
He_moles_melt_real_1xsolar = pd.read_excel(filename, sheet_name="element_He")["dissolved_moles"]
O_moles_melt_real_1xsolar = pd.read_excel(filename, sheet_name="element_O")["dissolved_moles"] + pd.read_excel(filename, sheet_name="element_O")["condensed_moles"]
C_moles_melt_real_1xsolar = pd.read_excel(filename, sheet_name="element_C")["dissolved_moles"]
N_moles_melt_real_1xsolar = pd.read_excel(filename, sheet_name="element_N")["dissolved_moles"]
Si_moles_melt_real_1xsolar = pd.read_excel(filename, sheet_name="element_Si")["dissolved_moles"] + pd.read_excel(filename, sheet_name="element_Si")["condensed_moles"]

H_moles_tot_real_1xsolar = pd.read_excel(filename, sheet_name="element_H")["total_moles"]
He_moles_tot_real_1xsolar = pd.read_excel(filename, sheet_name="element_He")["total_moles"]
O_moles_tot_real_1xsolar = pd.read_excel(filename, sheet_name="element_O")["total_moles"]
C_moles_tot_real_1xsolar = pd.read_excel(filename, sheet_name="element_C")["total_moles"]
N_moles_tot_real_1xsolar = pd.read_excel(filename, sheet_name="element_N")["total_moles"]
Si_moles_tot_real_1xsolar = pd.read_excel(filename, sheet_name="element_Si")["total_moles"]

H2_fc_real_1xsolar = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_1xsolar = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_1xsolar = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_1xsolar = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_1xsolar = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_1xsolar = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_1xsolar = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_1xsolar = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_1xsolar = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_1xsolar = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_1xsolar = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 10x solar
init_metallicity = 10 


filename = f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_10xsolar = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

H_moles_atm_real_10xsolar = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_10xsolar = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_10xsolar = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_10xsolar = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_10xsolar = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_10xsolar = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_moles_melt_real_10xsolar = pd.read_excel(filename, sheet_name="element_H")["dissolved_moles"]
He_moles_melt_real_10xsolar = pd.read_excel(filename, sheet_name="element_He")["dissolved_moles"]
O_moles_melt_real_10xsolar = pd.read_excel(filename, sheet_name="element_O")["dissolved_moles"] + pd.read_excel(filename, sheet_name="element_O")["condensed_moles"]
C_moles_melt_real_10xsolar = pd.read_excel(filename, sheet_name="element_C")["dissolved_moles"]
N_moles_melt_real_10xsolar = pd.read_excel(filename, sheet_name="element_N")["dissolved_moles"]
Si_moles_melt_real_10xsolar = pd.read_excel(filename, sheet_name="element_Si")["dissolved_moles"] + pd.read_excel(filename, sheet_name="element_Si")["condensed_moles"]

H_moles_tot_real_10xsolar = pd.read_excel(filename, sheet_name="element_H")["total_moles"]
He_moles_tot_real_10xsolar = pd.read_excel(filename, sheet_name="element_He")["total_moles"]
O_moles_tot_real_10xsolar = pd.read_excel(filename, sheet_name="element_O")["total_moles"]
C_moles_tot_real_10xsolar = pd.read_excel(filename, sheet_name="element_C")["total_moles"]
N_moles_tot_real_10xsolar = pd.read_excel(filename, sheet_name="element_N")["total_moles"]
Si_moles_tot_real_10xsolar = pd.read_excel(filename, sheet_name="element_Si")["total_moles"]

H2_fc_real_10xsolar = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_10xsolar = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_10xsolar = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_10xsolar = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_10xsolar = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_10xsolar = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_10xsolar = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_10xsolar = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_10xsolar = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_10xsolar = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_10xsolar = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]

# 100x solar
init_metallicity = 100

filename = f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar_melt{round(mantle_melt_fraction*100)}.xlsx"

H2_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="H2_g")["pressure"]
H2O_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="H2O_g")["pressure"]
O2_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="O2_g")["pressure"]
SiH4_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="H4Si_g")["pressure"]
SiO_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="OSi_g")["pressure"]
CO2_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="CO2_g")["pressure"]
CO_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="CO_g")["pressure"]
CH4_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="CH4_g")["pressure"]
N2_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="N2_g")["pressure"]
NH3_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="H3N_g")["pressure"]
He_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="He_g")["pressure"]
tot_pressure_real_100xsolar = pd.read_excel(filename, sheet_name="atmosphere")["pressure"]

H_moles_atm_real_100xsolar = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"]
He_moles_atm_real_100xsolar = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"]
O_moles_atm_real_100xsolar = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"]
C_moles_atm_real_100xsolar = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"]
N_moles_atm_real_100xsolar = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"]
Si_moles_atm_real_100xsolar = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"]

H_moles_melt_real_100xsolar = pd.read_excel(filename, sheet_name="element_H")["dissolved_moles"]
He_moles_melt_real_100xsolar = pd.read_excel(filename, sheet_name="element_He")["dissolved_moles"]
O_moles_melt_real_100xsolar = pd.read_excel(filename, sheet_name="element_O")["dissolved_moles"] + pd.read_excel(filename, sheet_name="element_O")["condensed_moles"]
C_moles_melt_real_100xsolar = pd.read_excel(filename, sheet_name="element_C")["dissolved_moles"]
N_moles_melt_real_100xsolar = pd.read_excel(filename, sheet_name="element_N")["dissolved_moles"]
Si_moles_melt_real_100xsolar = pd.read_excel(filename, sheet_name="element_Si")["dissolved_moles"] + pd.read_excel(filename, sheet_name="element_Si")["condensed_moles"]

H_moles_tot_real_100xsolar = pd.read_excel(filename, sheet_name="element_H")["total_moles"]
He_moles_tot_real_100xsolar = pd.read_excel(filename, sheet_name="element_He")["total_moles"]
O_moles_tot_real_100xsolar = pd.read_excel(filename, sheet_name="element_O")["total_moles"]
C_moles_tot_real_100xsolar = pd.read_excel(filename, sheet_name="element_C")["total_moles"]
N_moles_tot_real_100xsolar = pd.read_excel(filename, sheet_name="element_N")["total_moles"]
Si_moles_tot_real_100xsolar = pd.read_excel(filename, sheet_name="element_Si")["total_moles"]

H2_fc_real_100xsolar = pd.read_excel(filename, sheet_name="H2_g")["fugacity_coefficient"]
H2O_fc_real_100xsolar = pd.read_excel(filename, sheet_name="H2O_g")["fugacity_coefficient"]
O2_fc_real_100xsolar = pd.read_excel(filename, sheet_name="O2_g")["fugacity_coefficient"]
SiH4_fc_real_100xsolar = pd.read_excel(filename, sheet_name="H4Si_g")["fugacity_coefficient"]
SiO_fc_real_100xsolar = pd.read_excel(filename, sheet_name="OSi_g")["fugacity_coefficient"]
CH4_fc_real_100xsolar = pd.read_excel(filename, sheet_name="CH4_g")["fugacity_coefficient"]
CO_fc_real_100xsolar = pd.read_excel(filename, sheet_name="CO_g")["fugacity_coefficient"]
CO2_fc_real_100xsolar = pd.read_excel(filename, sheet_name="CO2_g")["fugacity_coefficient"]
NH3_fc_real_100xsolar = pd.read_excel(filename, sheet_name="H3N_g")["fugacity_coefficient"]
N2_fc_real_100xsolar = pd.read_excel(filename, sheet_name="N2_g")["fugacity_coefficient"]
He_fc_real_100xsolar = pd.read_excel(filename, sheet_name="He_g")["fugacity_coefficient"]


### Plots

In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9)) = plt.subplots(3,3, figsize=(9,9), tight_layout='True')

ax1.plot(hmps, H_moles_atm_real_1xsolar, color='black', lw=2, ls= '-', label='H')
ax1.plot(hmps, He_moles_atm_real_1xsolar, color='gray', lw=2, ls= '-', label='He')
ax1.plot(hmps, C_moles_atm_real_1xsolar, color='blue', lw=2, ls= '-', label='C')
ax1.plot(hmps, N_moles_atm_real_1xsolar, color='green', lw=2, ls= '-', label='N')
ax1.plot(hmps, O_moles_atm_real_1xsolar, color='orange', lw=2, ls= '-', label='O')
ax1.plot(hmps, Si_moles_atm_real_1xsolar, color='red', lw=2, ls= '-', label='Si')

ax1.set_title(f'Envelope Moles (1x solar)')
ax1.set_ylim([1e20, 8e26])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'H mass fraction [%]', fontsize=14)
ax1.set_ylabel(r'Envelope Models', fontsize=14)

ax2.plot(hmps, H_moles_atm_real_10xsolar, color='black', lw=2, ls= '-', label='H')
ax2.plot(hmps, He_moles_atm_real_10xsolar, color='gray', lw=2, ls= '-', label='He')
ax2.plot(hmps, C_moles_atm_real_10xsolar, color='blue', lw=2, ls= '-', label='C')
ax2.plot(hmps, N_moles_atm_real_10xsolar, color='green', lw=2, ls= '-', label='N')
ax2.plot(hmps, O_moles_atm_real_10xsolar, color='orange', lw=2, ls= '-', label='O')
ax2.plot(hmps, Si_moles_atm_real_10xsolar, color='red', lw=2, ls= '-', label='Si')

ax2.set_title(f'Envelope Moles (10x solar)')
ax2.set_ylim([1e20, 8e26])
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel(r'H mass fraction [%]', fontsize=14)
ax2.set_ylabel(r'Envelope Moles', fontsize=14)

ax3.plot(hmps, H_moles_atm_real_100xsolar, color='black', lw=2, ls= '-', label='H')
ax3.plot(hmps, He_moles_atm_real_100xsolar, color='gray', lw=2, ls= '-', label='He')
ax3.plot(hmps, C_moles_atm_real_100xsolar, color='blue', lw=2, ls= '-', label='C')
ax3.plot(hmps, N_moles_atm_real_100xsolar, color='green', lw=2, ls= '-', label='N');
ax3.plot(hmps, O_moles_atm_real_100xsolar, color='orange', lw=2, ls= '-', label='O');
ax3.plot(hmps, Si_moles_atm_real_100xsolar, color='red', lw=2, ls= '-', label='Si');
ax3.set_title(f'Envelope Moles (100x solar)')
ax3.set_ylim([1e20, 8e26])
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlabel(r'H mass fraction [%]', fontsize=14)
ax3.set_ylabel(r'Envelope Moles', fontsize=14)       

ax4.plot(hmps, H_moles_melt_real_1xsolar, color='black', lw=2, ls= '-', label='H')
ax4.plot(hmps, He_moles_melt_real_1xsolar, color='gray', lw=2, ls= '-', label='He')
ax4.plot(hmps, C_moles_melt_real_1xsolar, color='blue', lw=2, ls= '-', label='C')
ax4.plot(hmps, N_moles_melt_real_1xsolar, color='green', lw=2, ls= '-', label='N')
ax4.plot(hmps, O_moles_melt_real_1xsolar, color='orange', lw=2, ls= '-', label='O')
ax4.plot(hmps, Si_moles_melt_real_1xsolar, color='red', lw=2, ls= '-', label='Si')

ax4.set_title(f'Melt Moles (1x solar)')
ax4.set_ylim([1e20, 8e26])
ax4.set_xscale('log')
ax4.set_yscale('log')
ax4.set_xlabel(r'H mass fraction [%]', fontsize=14)
ax4.set_ylabel(r'Melt Moles', fontsize=14)

ax5.plot(hmps, H_moles_melt_real_10xsolar, color='black', lw=2, ls= '-', label='H')
ax5.plot(hmps, He_moles_melt_real_10xsolar, color='gray', lw=2, ls= '-', label='He')
ax5.plot(hmps, C_moles_melt_real_10xsolar, color='blue', lw=2, ls= '-', label='C')
ax5.plot(hmps, N_moles_melt_real_10xsolar, color='green', lw=2, ls= '-', label='N')
ax5.plot(hmps, O_moles_melt_real_10xsolar, color='orange', lw=2, ls= '-', label='O')
ax5.plot(hmps, Si_moles_melt_real_10xsolar, color='red', lw=2, ls= '-', label='Si')

ax5.set_title(f'Melt Moles (10x solar)')
ax5.set_ylim([1e20, 8e26])
ax5.set_xscale('log')
ax5.set_yscale('log')
ax5.set_xlabel(r'H mass fraction [%]', fontsize=14)
ax5.set_ylabel(r'Melt Moles', fontsize=14)

ax6.plot(hmps, H_moles_melt_real_100xsolar, color='black', lw=2, ls= '-', label='H')
ax6.plot(hmps, He_moles_melt_real_100xsolar, color='gray', lw=2, ls= '-', label='He')
ax6.plot(hmps, C_moles_melt_real_100xsolar, color='blue', lw=2, ls= '-', label='C')
ax6.plot(hmps, N_moles_melt_real_100xsolar, color='green', lw=2, ls= '-', label='N')
ax6.plot(hmps, O_moles_melt_real_100xsolar, color='orange', lw=2, ls= '-', label='O')
ax6.plot(hmps, Si_moles_melt_real_100xsolar, color='red', lw=2, ls= '-', label='Si')

ax6.set_title(f'Melt Moles (100x solar)')
ax6.set_ylim([1e20, 8e26])
ax6.set_xscale('log')
ax6.set_yscale('log')
ax6.set_xlabel(r'H mass fraction [%]', fontsize=14)
ax6.set_ylabel(r'Melt Moles', fontsize=14)

ax7.plot(hmps, H_moles_tot_real_1xsolar, color='black', lw=2, ls= '-', label='H')
ax7.plot(hmps, He_moles_tot_real_1xsolar, color='gray', lw=2, ls= '-', label='He')
ax7.plot(hmps, C_moles_tot_real_1xsolar, color='blue', lw=2, ls= '-', label='C')
ax7.plot(hmps, N_moles_tot_real_1xsolar, color='green', lw=2, ls= '-', label='N')
ax7.plot(hmps, O_moles_tot_real_1xsolar, color='orange', lw=2, ls= '-', label='O')
ax7.plot(hmps, Si_moles_tot_real_1xsolar, color='red', lw=2, ls= '-', label='Si')

ax7.set_title(f'Total Moles (1x solar)')
ax7.set_ylim([1e20, 8e26])
ax7.set_xscale('log')
ax7.set_yscale('log')
ax7.set_xlabel(r'H mass fraction [%]', fontsize=14)
ax7.set_ylabel(r'Total Moles', fontsize=14)

ax8.plot(hmps, H_moles_tot_real_10xsolar, color='black', lw=2, ls= '-', label='H')
ax8.plot(hmps, He_moles_tot_real_10xsolar, color='gray', lw=2, ls= '-', label='He')
ax8.plot(hmps, C_moles_tot_real_10xsolar, color='blue', lw=2, ls= '-', label='C')
ax8.plot(hmps, N_moles_tot_real_10xsolar, color='green', lw=2, ls= '-', label='N')
ax8.plot(hmps, O_moles_tot_real_10xsolar, color='orange', lw=2, ls= '-', label='O')
ax8.plot(hmps, Si_moles_tot_real_10xsolar, color='red', lw=2, ls= '-', label='Si')

ax8.set_title(f'Total Moles (10x solar)')
ax8.set_ylim([1e20, 8e26])
ax8.set_xscale('log')
ax8.set_yscale('log')
ax8.set_xlabel(r'H mass fraction [%]', fontsize=14)
ax8.set_ylabel(r'Total Moles', fontsize=14)

ax9.plot(hmps, H_moles_tot_real_100xsolar, color='black', lw=2, ls= '-', label='H')
ax9.plot(hmps, He_moles_tot_real_100xsolar, color='gray', lw=2, ls= '-', label='He')
ax9.plot(hmps, C_moles_tot_real_100xsolar, color='blue', lw=2, ls= '-', label='C')
ax9.plot(hmps, N_moles_tot_real_100xsolar, color='green', lw=2, ls= '-', label='N')
ax9.plot(hmps, O_moles_tot_real_100xsolar, color='orange', lw=2, ls= '-', label='O')
ax9.plot(hmps, Si_moles_tot_real_100xsolar, color='red', lw=2, ls= '-', label='Si')

ax9.set_title(f'Total Moles (100x solar)')
ax9.set_ylim([1e20, 8e26])
ax9.set_xscale('log')
ax9.set_yscale('log')
ax9.set_xlabel(r'H mass fraction [%]', fontsize=14)
ax9.set_ylabel(r'Total Moles', fontsize=14)

custom_lines = [Line2D([0], [0], color='black', lw=2),
                Line2D([0], [0], color='red', lw=2)]

ax2.legend(ncol=3, borderpad=0.1, labelspacing=0.1, handletextpad=0.1)

plt.savefig(f"HHeCNOSi_moles.pdf", bbox_inches='tight')
plt.savefig(f"HHeCNOSi_moles.png", bbox_inches='tight')
plt.show()

## Envelope Elemental Abundances in units of Metallicity

In [ ]:
filename = f"HHeCNOSi_magma_sol_real_1xsolar_melt100.xlsx"

H_moles_atm_1xsolar_1wtH = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"][49]
He_moles_atm_1xsolar_1wtH = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"][49]
C_moles_atm_1xsolar_1wtH = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"][49]
N_moles_atm_1xsolar_1wtH = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"][49]
O_moles_atm_1xsolar_1wtH = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"][49]
Si_moles_atm_1xsolar_1wtH = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"][49]

Si_metallicity_1xsolar = Si_moles_atm_1xsolar_1wtH / H_moles_atm_1xsolar_1wtH / (10**(Si_logN - H_logN))
He_metallicity_1xsolar = He_moles_atm_1xsolar_1wtH / H_moles_atm_1xsolar_1wtH / (10**(He_logN - H_logN))
C_metallicity_1xsolar = C_moles_atm_1xsolar_1wtH / H_moles_atm_1xsolar_1wtH / (10**(C_logN - H_logN))
N_metallicity_1xsolar = N_moles_atm_1xsolar_1wtH / H_moles_atm_1xsolar_1wtH / (10**(N_logN - H_logN))
O_metallicity_1xsolar = O_moles_atm_1xsolar_1wtH / H_moles_atm_1xsolar_1wtH / (10**(O_logN - H_logN))

filename = f"HHeCNOSi_magma_sol_real_100xsolar_melt100.xlsx"

H_moles_atm_100xsolar_1wtH = pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"][49]
He_moles_atm_100xsolar_1wtH = pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"][49]
C_moles_atm_100xsolar_1wtH = pd.read_excel(filename, sheet_name="element_C")["atmosphere_moles"][49]
N_moles_atm_100xsolar_1wtH = pd.read_excel(filename, sheet_name="element_N")["atmosphere_moles"][49]
O_moles_atm_100xsolar_1wtH = pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"][49]
Si_moles_atm_100xsolar_1wtH = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"][49]

Si_metallicity_100xsolar = Si_moles_atm_100xsolar_1wtH / H_moles_atm_100xsolar_1wtH / (10**(Si_logN - H_logN))
He_metallicity_100xsolar = He_moles_atm_100xsolar_1wtH / H_moles_atm_100xsolar_1wtH / (10**(He_logN - H_logN))
C_metallicity_100xsolar = C_moles_atm_100xsolar_1wtH / H_moles_atm_100xsolar_1wtH / (10**(C_logN - H_logN))
N_metallicity_100xsolar = N_moles_atm_100xsolar_1wtH / H_moles_atm_100xsolar_1wtH / (10**(N_logN - H_logN))
O_metallicity_100xsolar = O_moles_atm_100xsolar_1wtH / H_moles_atm_100xsolar_1wtH / (10**(O_logN - H_logN))

print("1 wt% H and 1x solar")
print("H", H_logN)
print("He", round(He_logN + np.log10(He_metallicity_1xsolar),2))
print("C", round(C_logN + np.log10(C_metallicity_1xsolar),2))
print("N", round(N_logN + np.log10(N_metallicity_1xsolar),2))
print("O", round(O_logN + np.log10(O_metallicity_1xsolar),2))
print("Si", round(Si_logN + np.log10(Si_metallicity_1xsolar),2))
print("")
print("1 wt% H and 100x solar")
print("H", "12")
print("He", round(He_logN + np.log10(He_metallicity_100xsolar),2))
print("C", round(C_logN + np.log10(C_metallicity_100xsolar),2))
print("N", round(N_logN + np.log10(N_metallicity_100xsolar),2))
print("O", round(O_logN + np.log10(O_metallicity_100xsolar),2))
print("Si", round(Si_logN + np.log10(Si_metallicity_100xsolar),2))